# **About some q-stuff** $\def\NN{\mathbb{N}}\def\KK{\mathbb{K}}\def\cR{\mathcal{R}}\def\cB{\mathcal{B}}\def\qbinomb#1#2#3{\left[\begin{array}{c}#1\\#2\end{array}\right]_{#3}}\def\qbinom#1#2{\qbinomb{#1}{#2}{q}}$

Let set up some machinery to manipulate the operators and check some $q$-stuff

In [1]:
%display latex
import sys
sys.path.insert(0, "..") # pseries_basis is here
from pseries_basis import *

B = QBinomialBasis()
q = B.q

The sequences that we are interested in work on the polynomials on $q$. This means we have to consider the sequences 
$$\left(\KK(q)\right)^{\NN}.$$

In this ring of sequences, the basis are sequences themselves $((a_{n,k})_k)_n$:

## **The $q$-binomial**
Can we try to have a real $q$-basis? Using the $q$-binomial, we obtain the following:

In [2]:
show(B);
basis_matrix(B)

Basis of Sequences over Fraction Field of Univariate Polynomial Ring in q over Rational Field

[                        1                         1                         1                         1                         1]
[                        0                         1                     q + 1               q^2 + q + 1         q^3 + q^2 + q + 1]
[                        0                         0                         1               q^2 + q + 1 q^4 + q^3 + 2*q^2 + q + 1]
[                        0                         0                         0                         1         q^3 + q^2 + q + 1]
[                        0                         0                         0                         0                         1]

The variable `B` contains the basis of $q$-binomials, i.e., `B` is a sequence $B_k(n)$ for $k\in \NN$ where:
$$B_k(n) = \qbinom{n}{k}.$$

We have a basis that looks pretty similar to the original binomial basis we were considering before. In fact, in the same way the original binomial basis was a *factorial basis*, we have that the $q$-binomial basis is $q$-factorial, meaning that there are rational sequences $(a_k)_k, (b_k)_k \in \KK(q)$ such that
$$\qbinom{n}{k+1} = (a_{k+1}q^n + b_{k+1})\qbinom{n}{k}.$$

In this case, we have that 
$$a_k = \frac{q}{q^k(q^{k} - 1)},\qquad b_k = \frac{1}{1-q^{k}}.$$

In [3]:
all(
    B[k+1](n) == 
    ((q/q^(k+1)/(q^(k+1)-1))*q^n + (1/(1-q^(k+1))))*B[k](n) 
    for n in range(10) 
    for k in range(10)
)

True

### **Compatibility with the shift operator $n \rightarrow n+1$**

We can find in the literature that the $q$-binomial satisfies the following recurrence equation:
$$\qbinom{n+1}{k} = q^k\qbinom{n}{k} + \qbinom{n}{k-1}.$$
We can check this identity for the first terms of our $q$-binomial basis `B` with the following code:

In [4]:
all(
    B[k](n+1) == 
    q^k*B[k](n) + B[k-1](n)
    for n in range(10)
    for k in range(1,10)
)

True

In the terms of compatibility, we have then that the $q$-binomial basis is compatible with $E_n: n \mapsto n+1$ with the compatiblity equation:
$$E_n \cdot B_k(n) = B_k(n+1) = q^k P_k(n) + P_{k-1}(n)$$
And if we write this in the recurrence form, we get:

$$\cR(E_n) = q^k + S_k$$

In [5]:
B.compatibility("E")

Compatibility condition (1, 0, 1) with following coefficient matrix:
[  1 q_k]

In [6]:
B.recurrence("E", output="ore_double")

Sk + q_k

In [7]:
B.compatibility("q_n**2")

Compatibility condition (0, 2, 1) with following coefficient matrix:
[                                   q_k^2         (q^2 + q)*q_k^3 + (-q - 1)*q_k^2 q^4*q_k^4 + (-q^3 - q^2)*q_k^3 + q*q_k^2]

In [8]:
check_compatibility(B, "E", lambda seq : seq.shift(), 30)

True

### **Compatibility with the multiplication by $q$**

This is trivial since $q$ is part of the field we are taking the sequences from. Hence the compatibility is trivial:
$$q B_k(n)$$

### **Compatibility with the multiplication by $q^n$**

By the definition of a $q$-factorial basis, all of these are (0,1)-compatible with the multiplication by $q^n$:
$$\qbinom{n}{k+1} = (a_kq^n + b_k)\qbinom{n}{k} \Longrightarrow q^n \qbinom{n}{k} = \frac{1}{a_k}\qbinom{n}{k+1} - \frac{b_k}{a_k}\qbinom{n}{k}$$

In this case, we have the following compatibility identity:
$$q^n \qbinom{n}{k} = q^k\qbinom{n}{k} + q^k(q^{k+1} - 1)\qbinom{n}{k+1}.$$

In [9]:
all(
    q^n * B[k](n)
    ==
    q^k * B[k](n) + q**(k)*(q**(k+1) - 1)*B[k+1](n)
    for n in range(10)
    for k in range(10)
)

True

Writing this in a different fashion, we get:

$$ Q \cdot B_k(n) = (q^n)B_k(n) = q^k B_k(n) + q^k(q^{k+1}-1)B_{k+1}(n)$$

Or in a recurrence way, we can write:

$$ \cR(Q) = q^k + q^{k-1}(q^k - 1)S_k^{-1} = q^k + q^{-1}(q^{2k} - q^k)S_k^{-1} = q^k + \frac{q^k(q^k - 1)}{q}S_k^{-1}$$

In [10]:
B.compatibility("q_n")

Compatibility condition (0, 1, 1) with following coefficient matrix:
[          q_k q*q_k^2 - q_k]

In [11]:
B.recurrence("q_n")

(1/q*q_k^2 - 1/q*q_k)*Ski + q_k

In [13]:
check_compatibility(B, "q_n", lambda seq : Qn*seq, 30)

True

## **Adapting this context to the `ore_algebra` package**

We can use the package `ore_algebra` to represent the operators $E$, $q_n$ and then compute the associated recurrences from any linear operator composed by $E$ and $q_n$. 

For these operators, we have the following relation among them:
* Multiplication by the constant sequence $(q)_n$: $q: (a_n)_n \mapsto (qa_n)_n$.
* Multiplication by the power sequence $(q^n)_n$: $q_n: (a_n)_n \mapsto (q^na_n)_n$.
* Shift operator w.r.t. the $n$: $E_n: (a_n)_n \mapsto (a_{n+1})_n$. We will also need the inverse of this operator $E_n^{-1}$.

We can use the implementation of the $q$-shift in `ore_algebra` to represent these operators. The method `get_qshift_algebra` will create these recurrences rings when given the names for the generators:

In [14]:
from pseries_basis.misc.ore import get_qshift_algebra
OE, (q_n, E) = get_qshift_algebra("q_n", "q", "E", rational=False, base=B.base)
show(tuple([OE, q_n, E]))

(Univariate Ore algebra in E over Univariate Polynomial Ring in q_n over Fraction Field of Univariate Polynomial Ring in q over Rational Field,
 q_n,
 E)

At this point, we can construct the original operator $L$ as an element of the ring `OE`. When we start, the operator $q_n$ is the multiplication by `q^n`, while once we obtain the compatible operator, it means the multiplication by `q^k`. Something similar happens to `E`. Berfore the compatibility it is the shift w.r.t. `n` but after the compatibility transformation, it is the shift w.r.t. `k`.

* To obtain the compatible operator: ``B.recurrence(L)``
* To apply an operator to a sequence: ``L.apply(sequence, actions)``
* To obtain the solution for an operator: ``solution(L, init)`` where init is a list (with enough elements - see method `pseries_basis.misc.ore.required_init`).

In [15]:
L = q_n*E^2 - q_n/q*E - 1

In [16]:
B.recurrence(L,output="ore_double")

q_k*Sk^2 + (((q^2 + q + 1)/q)*q_k^2 - 2/q*q_k)*Sk + (1/q^3*q_k^4 - 2/q^3*q_k^3 + 1/q^3*q_k^2)*Ski + ((q^2 + q + 1)/q^2)*q_k^3 + ((-2*q - 2)/q^2)*q_k^2 + 1/q^2*q_k - 1

## **Basic examples** (<span style="color:green">TODO: review this section</span>)

### The $q$-Pochhammer symbol (Theorem 3.3 - first identity)
    
Let us consider the $q$-polynomials that appear from the $q$-Pochhammer symbol (i.e., the correposnding to the falling factorial):

In [17]:
from pseries_basis.qbasis.qbasis import qpochhammer
C.<z> = B.base[]
BwZ = B.change_base(C)
q = BwZ.base('q'); z = BwZ.base('z')
zn = BwZ.QPochhammer(z)
zn[:4]

ImportError: cannot import name 'qpochhammer' from 'pseries_basis.qbasis.qbasis' (/home/qh82kz/git/pseries_basis/notebooks/../pseries_basis/qbasis/qbasis.py)

The variable `zn` contains the sequence $(z)_n = (z;q)_n$. By its simple definition, this sequence satisfies a very simple $q$-recurrence equation:
$$(z;q)_{n+1} = (1-zq^{n})(z;q)_n,$$
which, in terms of our operators mean:
$$L \cdot (z;q)_n = (E_n - 1 + zQ)\cdot (z;q)_n = 0$$

In [18]:
OEz, (q_n, E) = get_qshift_algebra("q_n", "q", "E", rational=False, base=BwZ.base)
L = E - 1 + z*q_n; L


E + z*q_n - 1

Since the $q$-shift $E_n$ and the multiplication by $q^n$ are compatible with the $q$-binomial basis, we can compute a $q$-recurrence equation when we write:
$$(z;q)_n = \sum_{k=0}^n c_k \qbinom{n}{k},$$
for the sequence $(c_k)_k$. This recurrence equation is obtained by substituing each operator in $L$ with its compatibility conditions:

In [19]:
L_comp = BwZ.recurrence(L)
show(L_comp)
cn = solution(L_comp, BwZ.functional_to_self(zn, required_init(L_comp)))
cn[:5]

Sn + (1/q*z*q_n^2 - 1/q*z*q_n)*Sni + (z + 1)*q_n - 1

[1, -z, q*z^2, -q^3*z^3, q^6*z^4]

We can check that this recurrence is satisfied by the sequence 
$$c_n = (-1)^n z^n q^{(n^2-n)/2}$$

(Note: we would need to implement closure properties to these operators in order to check symbolically and not only check the first terms)

In [20]:
cn_real = LambdaSequence(lambda n : (-1)**n * z**n * q**((n**2-n)/2), BwZ.base)
cn.almost_equals(cn_real, 50)

True

### Same sequence, different basis (Theorem 3.3 - second identity) (<span style="color:red">Not yet working</span>)

The second part of Theorem 3.3 is the following identity:
$$(z;q)_n = \sum_{j=0}^n z^j \qbinom{n+j+1}{j}.$$

Using the class `QBinomialBasis` we can still prove this identity:

### A more ellaborate identity (Theorem 3.4 - identity 3.3.8)

Consider now the following sequence:
$$a_m = \left\{\begin{array}{ll}
    (q;q^2)_n & \text{if }m = 2n\\
    0 & \text{otherwise}
\end{array}\right.$$

In [22]:
q = B.q()
am = InterlacingSequence(QPochhammerSequence(q, q^2), LambdaSequence(lambda n : 0, ZZ, 1))
am[:5]

[1, 0, -q + 1, 0, q^4 - q^3 - q + 1]

Now, we want to write this sequence using the $q$-binomial basis, in the following way:
$$a_m = \sum_{k = 0}^m c_k \qbinom{m}{k}.$$

If we manage to obtain an operator $L$ that annihilates $a_m$, then we can compute an operator that annihilates $(c_k)_k$.

This operator $L$ can be computed easily from the definition of $a_m$, obtaining:
$$L = E_m^2 - 1 + qQ.$$
We check this using the following code:

In [23]:
OE, (q_n, E) = get_qshift_algebra("q_n", "q", "E", rational=False, base=B.base)
L = E^2 - 1 + q*q_n
apply_operator_to_seq(L, am).almost_zero(50)

True

Hence we can compute the recursion for $(c_k)_k$ by substituing the compatibility operators into $L$:

$$-1 + q Q + Q^{2} + \left(q + 1\right) Q \mathit{Sn} - Q \mathit{Sni} + \mathit{Sn}^{2} + Q^{2}\mathit{Sni}$$

In [24]:
L_comp = B.recurrence(L)
show(L_comp)
ck = solution(L_comp, B.functional_to_self(am, required_init(L_comp)))
ck[:10]

Sn^2 + ((q + 1)*q_n)*Sn + (q_n^2 - q_n)*Sni + q_n^2 + q*q_n - 1

[1, -1, 1, -1, 1, -1, 1, -1, 1, -1]

In the theorem 3.4, indentity 3.3.8, they claim the sequence $c_k = (-1)^k$. Using closure properties over $L_{comp}$ and the annihilator of $(-1)^k$ (which would be $S + 1$) we could prove symbolically that these two sequence are the same. However, we will simply check that $(-1)^k$ isannihilated by our $L_{comp}$: 

In [25]:
ck_real = LambdaSequence(lambda k : (-1)**k, B.base)
ck.almost_equals(ck_real, 100)

True

### A weird binomial identity (Theorem 3.4 - identity 3.3.9) (<span style="color:red">Not yet working</span>)

In this example, we have the following identity:
$$\qbinom{n+m+1}{m+1} = \sum_{j=0}^n q^j\qbinom{m+j}{m}.$$

This identity looks like something we can prove using our code. However, we have a couple of issues with this:

* The $q$-binomial used in this identity do not have the summing parameter in the bottom: we need to adapt the compatibilities for this type of basis.
* $n$ is a parameter. Hence we are going to add two parameters to the base field: $n$ and $q^n$.

Let us start by obtaining a valid field for this identity and then, we will compute a recurrence that annihilates the left-hand side of the identity:

## **Examples from Ali** (<span style="color:green">TODO: review this section</span>)

### Example 25/05/2023

On May 25th, Ali came back to me with a new example for which he have a solution but want to try if the software is able to solve it. The recurrence is the follosing:
$$a_n = -q^{n-1}a_{n-1} + q^n a_{n-2},\quad a_0 = 2,\quad a_1 = q,$$
or equivalently:
$$a_{n+2} + q^{n+1}a_{n+1} - q^{n+2}a_n = 0.$$
First of all, let us compute the first elements of this sequence using the the basic software:

In [2]:
OQ, (qn, E) = get_qshift_algebra("qn", base=QQ["q"])
q = OQ.base().base().base()("q")

In [3]:
op = E^2 + q*qn*E - q^2*qn
sol = solution(op, [2, q], name_q = "q"); sol[:6]

[2, q, q^2, 0, q^6, -q^10]

Let us consider the basis generated by $\qbinom{2n}{k}$, and let us apply our machinery to the previously defined sequence $(a_n)_n$:

In [4]:
B = QBinomialBasis(2)
B.basic_compatibilities()

['q_2n', 'E']

In order to apply to $(a_n)_n$, we need to find an annihilator of $(a_n)_n$ that only contains $E$ and $q^{2n}$:

In [5]:
op

E^2 + q*qn*E - q^2*qn

In [6]:
new_op = (op - (q^3+q)*qn*E + (q^4 + q^2)*qn)*op; new_op

E^4 - q^5*qn^2*E^2 + ((q^6 + q^5)*qn^2)*E - q^6*qn^2

In [7]:
apply_operator_to_seq(new_op, sol)[:10] # 'sol' is solution to 'new_op'

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Now, we can apply the `recurrence` method to the new operator that also annihilates our sequence $(a_n)_n$:

In [8]:
rec = B.recurrence(str(new_op).replace("qn^2", "q_2n"), output="ore"); rec

Sk^9 + ((q^8 + q^7 + q^6 + q^5 + q^4 + q^3 + q^2 + q)*q_k)*Sk^8 + ((q^14 + q^13 + 2*q^12 + 2*q^11 + 3*q^10 + 3*q^9 + 4*q^8 + 3*q^7 + 3*q^6 + 2*q^5 + 2*q^4 + q^3 + q^2)*q_k^2)*Sk^7 + ((q^18 + q^17 + 2*q^16 + 3*q^15 + 4*q^14 + 5*q^13 + 6*q^12 + 6*q^11 + 6*q^10 + 6*q^9 + 5*q^8 + 4*q^7 + 3*q^6 + 2*q^5 + q^4 + q^3)*q_k^3)*Sk^6 + ((q^20 + q^19 + 2*q^18 + 3*q^17 + 5*q^16 + 5*q^15 + 7*q^14 + 7*q^13 + 8*q^12 + 7*q^11 + 7*q^10 + 5*q^9 + 5*q^8 + 3*q^7 + 2*q^6 + q^5 + q^4)*q_k^4 - q^6*q_k)*Sk^5 + ((q^20 + q^19 + 2*q^18 + 3*q^17 + 4*q^16 + 5*q^15 + 6*q^14 + 6*q^13 + 6*q^12 + 6*q^11 + 5*q^10 + 4*q^9 + 3*q^8 + 2*q^7 + q^6 + q^5)*q_k^5 + (-q^10 - q^9 - q^8 - q^7 - q^6)*q_k^2 + q^5*q_k)*Sk^4 + ((q^18 + q^17 + 2*q^16 + 2*q^15 + 3*q^14 + 3*q^13 + 4*q^12 + 3*q^11 + 3*q^10 + 2*q^9 + 2*q^8 + q^7 + q^6)*q_k^6 + (-q^12 - q^11 - 2*q^10 - 2*q^9 - 2*q^8 - q^7 - q^6)*q_k^3 + (q^8 + q^7 + q^6 + q^5)*q_k^2 + (q^7 + q^6)*q_k)*Sk^3 + ((q^14 + q^13 + q^12 + q^11 + q^10 + q^9 + q^8 + q^7)*q_k^7 + (-q^12 - q^11 - 2*q^10 - 2*q^9 - 2*q^8 - q^7 - q^6)*q_k^4 + (q^9 + q^8 + 2*q^7 + q^6 + q^5)*q_k^3 + (q^9 + 2*q^8 + 2*q^7 + q^6)*q_k^2 + (-q^6 - q^5)*q_k)*Sk^2 + (q^8*q_k^8 + (-q^10 - q^9 - q^8 - q^7 - q^6)*q_k^5 + (q^8 + q^7 + q^6 + q^5)*q_k^4 + (q^9 + 2*q^8 + 2*q^7 + q^6)*q_k^3 + (-q^7 - 2*q^6 - q^5)*q_k^2 - q^7*q_k)*Sk - q^6*q_k^6 + q^5*q_k^5 + (q^7 + q^6)*q_k^4 + (-q^6 - q^5)*q_k^3 - q^7*q_k^2 + q^6*q_k

To conclude the proccess, we would need to compute the initial terms of the new sequence. In theory, we are claiming that there is a solution $(c_k)_k$ to `rec` such that
$$a_n = \sum_{k\geq 0} c_k\qbinom{2n}{k}.$$
We would like to get a full grasp of the sequence $(c_k)_k$, so we need to compute the initial terms $(c_0,\ldots,c_8)$ that fully defines a solution to `rec`. Our main problem is that the basis we are considering is not such basis. We can see this in the matrix that is defined by the basis:

In [9]:
P = B.inner_init_values(sol, 10, rec)

In [10]:
C = solution(rec, P);
New_Sol = Sequence(lambda n : sum(C[k]*B[k][n] for k in range(B.is_quasi_triangular()[n]+1)), sol.universe)
all(el == 0 for el in (New_Sol - sol)[:50])

True

Now, in the original example, we want to find an expression of the form
$$a_n = \sum_{k=-\infty}^\infty \sum_{i=0}^4 \alpha_{k,i} \qbinom{2n}{n+5k+i}$$

To do this, we need to manipulate the right-hand side to fit our theory. First of all, let us consider the case with no distinction between the 5 sections:
$$a_n = \sum_{k=-\infty}^\infty c_k \qbinom{2n}{n+k}.$$
Now, we do the change of variables $n+k \mapsto k$. In this way, the sum from $-\infty$ to $\infty$ changes to the usual shape and the argument at the bottom of the $q$-binomial becomes simply $k$. Everything looks good so far, but now the coefficient in front of the $q$-binomial depends on $n$ and $k$ (not just $k$):
$$a_n =\sum_{k=0}^{\infty} c_{n-k} \qbinom{2n}{k}.$$

In order to continue, we need to know a bit of the shape of $c_{n-k}$. More precisely, we need to know the influence of $n$. Let us assume that we can write 
$$c_{n-k} = q^{n^2}q^{2kn}d_k.$$
Then, we can move the term only using $n$ to the left hand side and have a problem in our theory using Product bases:
$$a_nq^{-n^2} = \sum_{k=0}^\infty d_k q^{2nk} \qbinom{2n}{k}.$$

**Remark** If the term involving just $n$ is like $q^{\alpha n^2}$ this also works.

**Remark 2** We need to compute the recurrence for $\tilde{a}_n = a_nq^{-n^2}$ using only $q^{2n}$. 

**Remark 3** Can we change the product basis part and have $q^{\beta nk}$?

In [9]:
alpha = 1; beta = 2 # parameters of the remarks
P = QPowerBasis(beta)
Q = QBasis(
    lambda k : P[k//2 + k%2]*B[k//2], 
    P.base)
q=Q.q; q_k = B.compatibility("q_2n").change_base(Q.base)[0,0].extra_info()["variables"][0]

##### Compatibility with $q^{2n}$

In [10]:
show(P.compatibility("q_2n"))
show("-------------------------------------------------------------------------------------------")
show(B.compatibility("q_2n"))

Compatibility condition (0, 1, 1) with following coefficient matrix:
[0 1]

'-------------------------------------------------------------------------------------------'

Compatibility condition (0, 1, 1) with following coefficient matrix:
[          q_k q*q_k^2 - q_k]

In [11]:
Q.set_compatibility("q_2n", Compatibility([
    [P.compatibility("q_2n").change_base(Q.base)[0,0], P.compatibility("q_2n").change_base(Q.base)[0,1]],
    [B.compatibility("q_2n").change_base(Q.base)[0,0], B.compatibility("q_2n").change_base(Q.base)[0,1]]], 
    0, 1, 2), True, "any")

Compatibility condition (0, 1, 2) with following coefficient matrix:
[            0             1]
[          q_k q*q_k^2 - q_k]

In [12]:
check_compatibility(Q, "q_2n", lambda seq : Qn**2*seq, 30)

True

##### Compatibility with $E$

In [13]:
show(P.compatibility("E"))
show("-------------------------------------------------------------------------------------------")
show(B.compatibility("E"))

Compatibility condition (0, 0, 1) with following coefficient matrix:
[q_k^2]

'-------------------------------------------------------------------------------------------'

Compatibility condition (2, 0, 1) with following coefficient matrix:
[              1 ((q + 1)/q)*q_k           q_k^2]

If we just look to these two compatibilities, we can deduce the final compatibility:

In [14]:
A = P.compatibility("E")[(0,0)]; D = B.compatibility("E")[(0,-1)]; F = B.compatibility("E")[(0,0)]
G = Q.compatibility("q_2n")[(1,0)]; H = Q.compatibility("q_2n")[(1,1)]
coeffs = [
    [
        QRationalSequence(0, variables=[q_k], universe=Qn.universe, q=Qn.q),
        A*G.shift(-2),
        A*H.shift(-2),
        A*D,
        A*F
    ],
    [
        A.shift()*G.shift(-2)*G.shift(-2),
        A.shift()*G.shift(-2)*H.shift(-2),
        A.shift()*(H.shift(-2) + D*G.shift(-1)),
        A.shift()*D*H.shift(-1),
        A.shift()*F
    ]
]

In [15]:
Q.set_homomorphism("E", Compatibility(coeffs, 4, 0, 2), True)

Compatibility condition (4, 0, 2) with following coefficient matrix:
[                              0                     1/q^2*q_k^3       1/q^3*q_k^4 - 1/q^2*q_k^3               ((q + 1)/q)*q_k^3                           q_k^4]
[                    1/q^2*q_k^4       1/q^3*q_k^5 - 1/q^2*q_k^4 ((q^2 + q + 1)/q)*q_k^4 - q_k^3  (q + 1)*q_k^5 + (-q - 1)*q_k^4                       q^2*q_k^4]

In [16]:
check_compatibility(Q, "E", lambda seq: seq.shift(), 30)

True

##### Continuing with the example

In [22]:
mq = sol.universe.gens()[0]
new_new_op = ((qn**6)*q**10)*E^4 - (qn**(4)*q**(3))*E^2 + ((1+q)*qn**(2))*E - 1
new_sol = solution(new_new_op, [sol[i]/mq^(i^2) for i in range(7)], check_init=False)

all(new_sol[i]*mq**(i^2) == sol[i] for i in range(50))

True

In [45]:
M = Q.recurrence(str(new_new_op).replace("qn^6", "q_2n^3").replace("qn^4", "q_2n^2").replace("qn^2", "q_2n"), sections=10)

In [47]:
[M[(i,0)].polynomial().monomials() for i in range(10)]

[[Sk, 1],
 [Sk, 1],
 [Sk, 1],
 [Sk, 1],
 [Sk],
 [Sk^2, Sk],
 [Sk^2, Sk],
 [Sk^2, Sk],
 [Sk^2, Sk],
 [Sk^2, Sk]]

### A new example

On 13/03/2023, Ali sent me the following recurrence equation:

$$\begin{array}{lrl}
    q^{12 + 15 n} (1 + q^{4 + 3 n} + q^{7 + 3 n}) & f(n) & +\\
    q^{10 + 9 n} \left(1 + q + q^{2 + 3 n} + q^{3 + 3 n} + q^{4 + 3 n} + q^{5 + 3 n} + q^{7 + 3 n} + q^{8 + 3 n} - q^{9 + 3 n} + q^{6 + 6 n} +
     q^{9 + 6 n}\right) & f(n+1) & + \\
     \left(-1 - q^{1 + 3 n} - q^{4 + 3 n} - q^{5 + 3 n} - q^{6 + 3 n} - q^{7 + 3 n} - q^{6 + 6 n} - q^{7 + 6 n} - q^{8 + 6 n} -
     q^{9 + 6 n} - q^{10 + 6 n} - q^{11 + 6 n} - 2 q^{12 + 6 n} - q^{10 + 9 n} - 2 q^{13 + 9 n} - q^{16 + 9 n}\right) & f(n+2) & + \\
     \left(1 + q^{1 + 3 n} + q^{4 + 3 n}\right) & f(n+3) & =0
\end{array}$$

$$f(0) = 0,\quad f(1) = 1+q,\quad f(2) = 1+q+q^2+2q^3 + 2q^4 + q^5 + 2q^6 + q^7$$

In [9]:
from pseries_basis.misc.ore import get_qshift_algebra
OE, (q_n, E, Ei) = get_double_qshift_algebra("q_3n", "q", "E", power=3, rational=False, base=B.base)
show(tuple([OE, q_n, E, Ei]))

(Multivariate Ore algebra in E, Ei over Univariate Polynomial Ring in q_3n over Fraction Field of Univariate Polynomial Ring in q over Rational Field,
 q_3n,
 E,
 Ei)

In [10]:
L = sum([
    q**12*q_n**5*(1+(q**4 + q**7)*q_n),
    q**10*q_n**3*(1+q+(q**2 + q**3 + q**4 + q**5 + q**7 + q**8 - q**9)*q_n + (q**6+q**9)*q_n**2)*E,
    (-1 - (q + q**4 + q**5 + q**6 + q**7)*q_n - (q**6 + q**7 + q**8 + q**9 + q**10 + q**11 + 2*q**12)*q_n**2 - (q**10 + 2*q**13 + q**16)*q_n**3)*E^2,
    (1 + q*q_n + q**3 * q_n)*E^3
]);L

((q^3 + q)*q_3n + 1)*E^3 + ((-q^16 - 2*q^13 - q^10)*q_3n^3 + (-2*q^12 - q^11 - q^10 - q^9 - q^8 - q^7 - q^6)*q_3n^2 + (-q^7 - q^6 - q^5 - q^4 - q)*q_3n - 1)*E^2 + ((q^19 + q^16)*q_3n^5 + (-q^19 + q^18 + q^17 + q^15 + q^14 + q^13 + q^12)*q_3n^4 + (q^11 + q^10)*q_3n^3)*E + (q^19 + q^16)*q_3n^6 + q^12*q_3n^5

In [12]:
sol = solution(L, [0, 1+q, 1+q+q^2+2*q^3 + 2*q^4 + q^5 + 2*q^6 + q^7]); sol

Sequence over [Fraction Field of Univariate Polynomial Ring in q over Rational Field]: (0, q + 1, q^7 + 2*q^6 + q^5 + 2*q^4 + 2*q^3 + q^2 + q + 1,...)

He wanted to try the following basis: 
$$\mathcal{B} = \left\{\left[\begin{array}{c}2n\\n+j\end{array}\right]_{q^3}\right\}_{j\in \mathbb{N}}.$$

Can we apply this software to get this? It seems this will leave to some problems:

* The fact that the base is $q^3$ and the coefficient on the top is $2n$ guarantees compatibility with $q^{6n}$. However, we have only $q^{3n}$.
* The lower coefficient involves $n$ and I do not know how to translate this into a compatibility.

Let me use the following two bases:
1. $P^{(1)}_j(n) = \left[\begin{array}{c}n\\j\end{array}\right]_{q^3}$.
2. $P^{(2)}_j(n) = \left[\begin{array}{c}3n\\j\end{array}\right]_{q}$.

#### **First basis: with the $q^3$**

We can write the compatibility conditions for this basis:

**Compatibility with $q^{3n}$:**

For this compatibility we simply use the formula for the $q$-binomial:
$$P^{(1)}_j(n) = \left[\begin{array}{c}n\\j\end{array}\right]_{q^3} = \prod_{i=0}^{j-1} \frac{1 - q^{3(n+i)}}{1 - q^{3(i+1)}},$$
hence we can write an easy formula when $j \mapsto j+1$:
$$P^{(1)}_{j+1}(n) = P^{(1)}_j(n)\frac{1-q^{3n}q^{3j}}{1 - q^{3j}q^3}.$$

Let us rewrite now this to see the compatibility with $q^{3n}$:
$$q^{3n}P^{(1)}_j(n) = \frac{1}{q^{3j}}P^{(1)}_j(n)-\frac{1 - q^{3j}q^3}{q^{3j}}P^{(1)}_{j+1}(n).$$

**Compatibility with the shift operator $n\mapsto n+1$**

For this compatibility we simply use the $q$-binomial coefficient identity:
$$P^{(1)}_j(n+1) = q^{3j}P^{(1)}_j(n) + P^{(1)}_{j-1}(n)$$

In [19]:
q_n, E = 1/q_n - q^3*(1-q_n)/q_n*Ei, q_n + E
L_1 = sum([
    q**12*q_n**5*(1+(q**4 + q**7)*q_n),
    q**10*q_n**3*(1+q+(q**2 + q**3 + q**4 + q**5 + q**7 + q**8 - q**9)*q_n + (q**6+q**9)*q_n**2)*E,
    (-1 - (q + q**4 + q**5 + q**6 + q**7)*q_n - (q**6 + q**7 + q**8 + q**9 + q**10 + q**11 + 2*q**12)*q_n**2 - (q**10 + 2*q**13 + q**16)*q_n**3)*E^2,
    (1 + q*q_n + q**3 * q_n)*E^3
])
q_n, E, Ei = gens_double_qshift_algebra(OE)[:3]

In [37]:
monomials, coefficients = L_1.polynomial().monomials(), L_1.polynomial().coefficients()
mon_degs = [m.degree(E.polynomial()) - m.degree(Ei.polynomial()) for m in monomials]

In [44]:
new_L = lcm([el.denominator() for el in coefficients])*sum(coeff*(E**deg if deg >=0 else Ei**(-deg)) for deg,coeff in zip(mon_degs, coefficients))

In [46]:
new_L.coefficients()[0]

(q^37 + q^34)*q_3n^6 + (-q^52 - 2*q^49 - 2*q^46 - 2*q^43 - 2*q^40 - 2*q^37 - q^34)*q_3n^5 + (q^64 + 2*q^61 + 3*q^58 + 4*q^55 + 5*q^52 + 5*q^49 + 4*q^46 + 3*q^43 + 2*q^40 + q^37)*q_3n^4 + (-q^73 - 2*q^70 - 3*q^67 - 5*q^64 - 6*q^61 - 6*q^58 - 6*q^55 - 5*q^52 - 3*q^49 - 2*q^46 - q^43)*q_3n^3 + (q^79 + 2*q^76 + 3*q^73 + 4*q^70 + 5*q^67 + 5*q^64 + 4*q^61 + 3*q^58 + 2*q^55 + q^52)*q_3n^2 + (-q^82 - 2*q^79 - 2*q^76 - 2*q^73 - 2*q^70 - 2*q^67 - q^64)*q_3n + q^82 + q^79

In [48]:
new_L.polynomial().monomials()

[Ei^6, Ei^5, Ei^4, E^3, Ei^3, E^2, Ei^2, E, Ei, 1]

#### **Second basis: with the $3n$**

We can write the compatibility conditions for this basis:

**Compatibility with $q^{3n}$:**

For this compatibility we simply use the formula for the $q$-binomial:
$$P^{(2)}_j(n) = \qbinom{3n}{j} = \prod_{i=0}^{j-1} \frac{1 - q^{3n+i}}{1 - q^{i+1}},$$
hence we can write an easy formula when $j \mapsto j+1$:
$$P^{(2)}_{j+1}(n) = P^{(2)}_j(n)\frac{1-q^{3n}q^{j}}{1 - q^{j}q} = P^{(2)}_j(n)\left(\frac{1}{1-qq^j} + q^{3n}\frac{q^j}{qq^j - 1}\right).$$

Let us rewrite now this to see the compatibility with $q^{3n}$:
$$q^{3n}P^{(2)}_j(n) = \frac{1}{q^{j}}P^{(2)}_j(n)-\frac{1 - q^{j}q}{q^{j}}P^{(2)}_{j+1}(n).$$

**Compatibility with the shift operator $n\mapsto n+1$**

For this compatibility we use the $q$-binomial coefficient identity, but we need to use three times:
$$\begin{array}{rl}
    P^{(2)}_j(n+1) = & \qbinom{3n+3}{j} \\
                   = & q^j\qbinom{3n+2}{j} + \qbinom{3n+2}{j-1} \\
                   = & q^{2j}\qbinom{3n+1}{j} + q^{j-1}(q + 1)\qbinom{3n+1}{j-1} + \qbinom{3n+1}{j-2}\\
                   = & q^{3j}\qbinom{3n}{j} + q^{2(j-1)}(q^2 + 1)(q+1)\qbinom{3n}{j-1} + q^{j-2}(q^2 + q + 1)\qbinom{3n}{j-2} + \qbinom{3n}{j-3}\\
                   = & q^{3j}P^{(2)}_j(n) + q^{2(j-1)}(q^2 + 1)(q+1)P^{(2)}_{j-1}(n) + q^{j-2}(q^2 + q + 1)P^{(2)}_{j-2}(n) + P^{(2)}_{j-3}(n)
\end{array}$$

In [59]:
## Trying to build the basis
from pseries_basis.qbasis.qbasis import QSFactorialBasis
q = QBinomialBasis().q()
an = q_n/(q*q_n-1)
bn = 1/(1-q*q_n)
def alpha(_,j,k):
    if j == 0: return q**k
    elif j == -1: return q**(2*(k-1))*(q**2 + 1)*(q + 1)
    elif j == -2: return q**(j-2)*(q**2+q+1)
    elif j == -3: return 1
B2 = QSFactorialBasis(an,bn,var_name="q_n")
B2.set_endomorphism("E", (3, 0, 1, alpha), False)

In [62]:
E.parent()

Univariate Ore algebra in E over Univariate Polynomial Ring in q_n over Fraction Field of Univariate Polynomial Ring in q over Rational Field

### Smaller example

Let us consider the solution of the $q$-recurrence equation:
$$qy_n = q^{2n+1}(1+q) y_{n-1} - q^{3n+1}(q^{n-1} - 1)y_{n-2}$$
We create now the corresponding operator and its solution $(y_n)_n$:

In [49]:
L = Q^3*(-q+Q)*Si^2 - q*Q^2*(1+q)*Si + q
yn = solution(L, [1, q^2*(1+q)])

In [27]:
yn[:5]

[1,
 q^3 + q^2,
 q^8 + q^7 + 2*q^6,
 q^15 + q^14 + 2*q^13 + 3*q^12 + q^11,
 q^24 + q^23 + 2*q^22 + 3*q^21 + 5*q^20 + 2*q^19 + 2*q^18]

In [28]:
Q

q_n

If we want to write now the solution:
$$y_n = \sum_{k=0}^n c_k \qbinom{n}{k},$$
then we can use the method `B.recurrence` to obtain a recurrence for $c_k$ and also obtain the first terms by using the method `B.funcitonal_to_self`.

In the following cell we compute the recurrence for $c_k$ and we display the degrees of this new recurrence with respect to $Q \equiv q^n$, $Sn$ and $Sn^{-1}$:

In [29]:
from pseries_basis.misc.ore import eval_ore_operator
L2 = B.recurrence(eval_ore_operator(B.remove_Sni(L), OE, Sn = E, q_n = q_n, Sni = 1))
show(L2)
L2.polynomial().degrees()

(1/q^8*q_n^8 + ((-q^3 - q^2 - q - 1)/q^8)*q_n^7 + ((q^4 + q^3 + 2*q^2 + q + 1)/q^7)*q_n^6 + ((-q^3 - q^2 - q - 1)/q^5)*q_n^5 + 1/q^2*q_n^4)*Sni^4 + (((q^3 + q^2 + q + 1)/q^4)*q_n^7 + ((-q^5 - 2*q^4 - 3*q^3 - 4*q^2 - 2*q - 1)/q^4)*q_n^6 + ((q^5 + 2*q^4 + 4*q^3 + 4*q^2 + 3*q + 1)/q^3)*q_n^5 + ((-q^3 - 2*q^2 - 2*q - 2)/q)*q_n^4 + q*q_n^3)*Sni^3 + q*Sn^2 + ((q^4 + q^3 + 2*q^2 + q + 1)*q_n^6 + ((-q^6 - 2*q^5 - 4*q^4 - 4*q^3 - 3*q^2 - 2*q - 1)/q)*q_n^5 + ((q^6 + 2*q^5 + 4*q^4 + 3*q^3 + 3*q^2 + 2*q + 1)/q)*q_n^4 + (-q^4 - q^3 - q^2 - q - 1)*q_n^3)*Sni^2 + ((-q^6 - q^5)*q_n^2 + (q^2 + q)*q_n)*Sn + ((q^7 + q^6 + q^5 + q^4)*q_n^5 + (-q^7 - 2*q^6 - 2*q^5 - 3*q^4 - 2*q^3 - 2*q^2 - q)*q_n^4 + (q^6 + q^5 + 2*q^4 + 3*q^3 + 3*q^2 + q)*q_n^3 + (-q^3 - q^2)*q_n^2)*Sni + q^8*q_n^4 + (-q^7 - q^6 - 2*q^5 - 2*q^4 - q^3)*q_n^3 + (q^5 + 2*q^4 + q^3 + q)*q_n^2

(2, 4)

We can then compute the solution $c_k$ using again the method `solution`:

In [30]:
ck = solution(L2, B.functional_to_self(yn, required_init(L2)))
for i in range(6):
    print(f"{i} -- {ck[i]}")

0 -- 1
1 -- q^3 + q^2 - 1
2 -- q^8 + q^7 + 2*q^6 - q^4 - 2*q^3 - q^2 + q
3 -- q^15 + q^14 + 2*q^13 + 3*q^12 + q^11 - q^10 - 2*q^9 - 4*q^8 - 3*q^7 - q^6 + 2*q^5 + 2*q^4
4 -- q^24 + q^23 + 2*q^22 + 3*q^21 + 5*q^20 + 2*q^19 + q^18 - 2*q^17 - 4*q^16 - 7*q^15 - 7*q^14 - 5*q^13 - 2*q^12 + 4*q^11 + 5*q^10 + 5*q^9 + q^8 - q^6 - q^5
5 -- q^35 + q^34 + 2*q^33 + 3*q^32 + 5*q^31 + 7*q^30 + 5*q^29 + 3*q^28 + q^27 - 3*q^26 - 7*q^25 - 12*q^24 - 13*q^23 - 13*q^22 - 10*q^21 - 4*q^20 + 5*q^19 + 10*q^18 + 13*q^17 + 12*q^16 + 6*q^15 + q^14 - 3*q^13 - 4*q^12 - 4*q^11 - 2*q^10 + q^8


### Old example (<span style="color:red">Not yet working</span>)

The example by Ali is a recurrence with plenty of components:

$$\begin{array}{rll}
q^{19 + 6n}\left({\left(q^{24} + q^{23} + q^{22}\right)} q^{6 \, n} - {\left(q^{20} + q^{19} + q^{18}\right)} q^{5 \, n} + {\left(q^{17} + q^{16} + q^{15}\right)} q^{4 \, n} + {\left(q^{12} + q^{11} + q^{10}\right)} q^{3 \, n} - {\left(q^{9} + q^{8} + q^{7}\right)} q^{2 \, n} + {\left(q^{5} + 2 \, q^{4} + q^{3}\right)} q^{n} - 1\right) & f(n) + &  \\
q^{5 \, n + 19}\left({\left(q^{28} + q^{27} + q^{26} + q^{25} + q^{24} + q^{23}\right)} q^{8 \, n} - {\left(q^{21} + q^{20} + q^{19}\right)} q^{7 \, n} + {\left(q^{21} + q^{20} + q^{19} + q^{18} + q^{17} + q^{16}\right)} q^{6 \, n} + {\left(q^{16} + q^{15} + q^{14} + q^{13} + q^{12} + q^{11}\right)} q^{5 \, n} + {\left(q^{15} + q^{14} + q^{13} - q^{10} - q^{9} - q^{8}\right)} q^{4 \, n} - {\left(q^{12} + q^{11} - q^{9} - 2 \, q^{8} - 2 \, q^{7} - 2 \, q^{6} - 2 \, q^{5} - q^{4}\right)} q^{3 \, n} + {\left(q^{8} - q^{5} - q\right)} q^{2 \, n} + {\left(q^{3} + q^{2} + q\right)} q^{n} - 1\right) & f(1 + n) + & \\
q^{3 \, n + 12} \left( {\left(q^{37} + q^{36} + q^{35}\right)} q^{11 \, n} - {\left(q^{33} + q^{32} + q^{31}\right)} q^{10 \, n} + {\left(q^{32} + 2 \, q^{31} + 3 \, q^{30} + 3 \, q^{29} + 2 \, q^{28} + q^{27}\right)} q^{9 \, n} - {\left(q^{30} + 2 \, q^{29} + 3 \, q^{28} + 2 \, q^{27} + 2 \, q^{26} - q^{23}\right)} q^{8 \, n} + {\left(q^{27} + 2 \, q^{26} + 3 \, q^{25} + 3 \, q^{24} + 2 \, q^{23} + q^{22} - q^{21} - q^{20} - q^{19}\right)} q^{7 \, n} - {\left(q^{25} + 2 \, q^{24} + 4 \, q^{23} + 2 \, q^{22} + 2 \, q^{21} - q^{20} - 2 \, q^{19} - 4 \, q^{18} - 4 \, q^{17} - 3 \, q^{16} - q^{15}\right)} q^{6 \, n} + {\left(q^{21} + 2 \, q^{20} + 2 \, q^{19} - q^{17} - 3 \, q^{16} - 2 \, q^{15} - 3 \, q^{14} - 2 \, q^{13} - q^{12}\right)} q^{5 \, n} - {\left(q^{18} + q^{17} + 2 \, q^{16} - q^{14} - 3 \, q^{13} - 3 \, q^{12} - 3 \, q^{11} - 2 \, q^{10} - q^{9}\right)} q^{4 \, n} - {\left(q^{13} + q^{12} + 3 \, q^{11} + 3 \, q^{10} + 3 \, q^{9} + q^{8} + q^{7}\right)} q^{3 \, n} + {\left(q^{10} + q^{9} + 2 \, q^{8} + 2 \, q^{7} + q^{6}\right)} q^{2 \, n} - {\left(q^{6} + 2 \, q^{5} + 3 \, q^{4} + 2 \, q^{3}\right)} q^{n} + q + 1 \right) & f(2 + n) + & \\
q^{2 \, n + 10} \left( {\left(q^{40} + q^{39} + q^{38}\right)} q^{12 \, n} - {\left(q^{37} + 2 \, q^{36} + 2 \, q^{35} + q^{34}\right)} q^{11 \, n} + {\left(q^{35} + q^{34} + 2 \, q^{33} + q^{32} + q^{31}\right)} q^{10 \, n} + {\left(q^{30} + 2 \, q^{29} + 3 \, q^{28} + 2 \, q^{27} + q^{26}\right)} q^{9 \, n} - {\left(q^{29} + q^{28} + 3 \, q^{27} + 4 \, q^{26} + 6 \, q^{25} + 5 \, q^{24} + 3 \, q^{23} + q^{22}\right)} q^{8 \, n} + {\left(q^{26} + 4 \, q^{25} + 5 \, q^{24} + 7 \, q^{23} + 7 \, q^{22} + 7 \, q^{21} + 5 \, q^{20} + 3 \, q^{19} + q^{18}\right)} q^{7 \, n} - {\left(2 \, q^{22} + 3 \, q^{21} + 3 \, q^{20} + 3 \, q^{19} + 2 \, q^{18} + 2 \, q^{17} + q^{16} + q^{15}\right)} q^{6 \, n} + {\left(q^{18} - q^{15} - 2 \, q^{14} - 2 \, q^{13} - q^{12}\right)} q^{5 \, n} + {\left(2 \, q^{15} + 4 \, q^{14} + 7 \, q^{13} + 7 \, q^{12} + 8 \, q^{11} + 6 \, q^{10} + 4 \, q^{9} + q^{8}\right)} q^{4 \, n} - {\left(q^{12} + 3 \, q^{11} + 5 \, q^{10} + 6 \, q^{9} + 5 \, q^{8} + 4 \, q^{7} + 2 \, q^{6} + q^{5}\right)} q^{3 \, n} + {\left(q^{8} + 2 \, q^{7} + 5 \, q^{6} + 5 \, q^{5} + 4 \, q^{4} + 2 \, q^{3} + q^{2}\right)} q^{2 \, n} - q^{n + 2} - 1 \right) & f(3 + n) + & \\
q^{n + 4} - 1 \left( {\left(q^{18} + q^{17} + q^{16}\right)} q^{6 \, n} - {\left(q^{15} + q^{14} + q^{13}\right)} q^{5 \, n} + {\left(q^{13} + q^{12} + q^{11}\right)} q^{4 \, n} + {\left(q^{9} + q^{8} + q^{7}\right)} q^{3 \, n} - {\left(q^{7} + q^{6} + q^{5}\right)} q^{2 \, n} + {\left(q^{4} + 2 \, q^{3} + q^{2}\right)} q^{n} - 1 \right) & f[4 + n] & = 0
\end{array}$$

In [51]:
L = (-Q^3*(q+Q)*(-q+Q)*(-q^3+Q^2)*(1-Q+Q^2)*Si^2) + (-q^2*Q^2*(q^2 + Q^3 - Q^5 + Q^6 + q^2*Q + q^3*Q - q*Q^2 - q^2*Q^2 + q*Q^3 + q*Q^4 +  q^2*Q^4 - q*Q^5)) + (q^2*(q^2 + Q^2 - q*Q))
B.remove_Sni(L)

(-q^18*q_n^8 + (q^17 + q^16)*q_n^7 + (-q^16 - q^15)*q_n^6 + (-q^13 - q^12)*q_n^5 + (q^12 + q^11)*q_n^4 + (-q^11 - q^10)*q_n^3 + (-q^8 + q^6)*q_n^2 - q^5*q_n + q^4)*Sn^2 - q^18*q_n^9 + q^16*q_n^8 + (q^17 + q^16 - q^14)*q_n^7 + (-q^15 - q^14)*q_n^6 + (-q^15 + q^13 + q^12)*q_n^5 + q^13*q_n^4 - q^11*q_n^3

In [53]:
L2 = B.remove_Sni(B.recurrence(eval_ore_operator(B.remove_Sni(L), OE, Sn = E, q_n = q_n, Sni = 1)))

In [57]:
print(L2.polynomial().coefficients()[0])

-q^99*q_n^18 + (q^98 + q^97 + q^96 + q^95 + q^94 + q^93 + q^92 + q^91 + q^90)*q_n^17 + (-q^96 - q^95 - 2*q^94 - 2*q^93 - 3*q^92 - 3*q^91 - 4*q^90 - 4*q^89 - 4*q^88 - 3*q^87 - 3*q^86 - 2*q^85 - 2*q^84 - q^83 - q^82)*q_n^16 + (q^93 + q^92 + 2*q^91 + 3*q^90 + 4*q^89 + 5*q^88 + 7*q^87 + 7*q^86 + 8*q^85 + 8*q^84 + 8*q^83 + 7*q^82 + 7*q^81 + 5*q^80 + 4*q^79 + 3*q^78 + 2*q^77 + q^76 + q^75)*q_n^15 + (-q^89 - q^88 - 2*q^87 - 3*q^86 - 5*q^85 - 6*q^84 - 8*q^83 - 9*q^82 - 11*q^81 - 11*q^80 - 12*q^79 - 11*q^78 - 11*q^77 - 9*q^76 - 8*q^75 - 6*q^74 - 5*q^73 - 3*q^72 - 2*q^71 - q^70 - q^69)*q_n^14 + (q^84 + q^83 + 2*q^82 + 3*q^81 + 5*q^80 + 6*q^79 + 8*q^78 + 9*q^77 + 11*q^76 + 11*q^75 + 12*q^74 + 11*q^73 + 11*q^72 + 9*q^71 + 8*q^70 + 6*q^69 + 5*q^68 + 3*q^67 + 2*q^66 + q^65 + q^64)*q_n^13 + (-q^78 - q^77 - 2*q^76 - 3*q^75 - 4*q^74 - 5*q^73 - 7*q^72 - 7*q^71 - 8*q^70 - 8*q^69 - 8*q^68 - 7*q^67 - 7*q^66 - 5*q^65 - 4*q^64 - 3*q^63 - 2*q^62 - q^61 - q^60)*q_n^12 + (q^71 + q^70 + 2*q^69 + 2*q^68 + 3*q^67 

In [26]:
L3 = ((S^9)*L2).canonical()

In [31]:
for k,v in L3.monomial_coefficients().items():
    print(f"{k} -- {v}")

Q^16*S^3 -- -q^114 - q^113 - 2*q^112 - 2*q^111 - 3*q^110 - 3*q^109 - 4*q^108 - 4*q^107 - 5*q^106 - 4*q^105 - 4*q^104 - 3*q^103 - 3*q^102 - 2*q^101 - 2*q^100 - q^99 - q^98
Q^17*S^2 -- -q^108 - q^107 - q^106 - q^105 - q^104 - q^103 - q^102 - q^101 - q^100 - q^99
Q^18*S -- -q^100
Q^15*S^3 -- -q^117 - q^116 - 2*q^115 - 3*q^114 - 4*q^113 - 5*q^112 - 7*q^111 - 6*q^110 - 6*q^109 - 4*q^108 - 2*q^107 + 2*q^106 + 5*q^105 + 12*q^104 + 18*q^103 + 22*q^102 + 25*q^101 + 27*q^100 + 27*q^99 + 27*q^98 + 25*q^97 + 22*q^96 + 17*q^95 + 12*q^94 + 9*q^93 + 6*q^92 + 4*q^91 + 2*q^90 + q^89
Q^16*S^2 -- -q^113 - q^112 - 2*q^111 - 2*q^110 - 3*q^109 - 3*q^108 - 4*q^107 - 4*q^106 - 3*q^105 - q^104 + 2*q^102 + 3*q^101 + 5*q^100 + 6*q^99 + 9*q^98 + 9*q^97 + 7*q^96 + 6*q^95 + 5*q^94 + 4*q^93 + 3*q^92 + 2*q^91 + q^90
Q^17*S -- -q^107 - q^106 - q^105 - q^104 - q^103 - q^102 - q^101 - q^100 - q^99 + q^98 + q^97 + q^96 + q^95 + q^94 + q^93 + q^92 + q^91
Q^15*S^4 -- -q^118 - q^117 - 2*q^116 - 3*q^115 - 4*q^114 - 5*q^113 -

In [24]:
q,n = var('q,n')
coeffs = [
    (-q^(19 + 6*n) + q^(22 + 7*n) + 2*q^(23 + 7*n) + q^(24 + 7*n) - q^(26 + 8*n) - q^(27 + 8*n) - q^(28 + 8*n) + q^(29 + 9*n) + # f(n)
     q^(30 + 9*n) + q^(31 + 9*n) + q^(34 + 10*n) + q^(35 + 10*n) + q^(36 + 10*n) - q^(37 + 11*n) - q^(38 + 11*n) - q^(39 + 11*n) + 
     q^(41 + 12*n) + q^(42 + 12*n) + q^(43 + 12*n)), 
   (-q^(19 + 5*n) + q^(20 + 6*n) + q^(21 + 6*n) + q^(22 + 6*n) - q^(20 + 7*n) - q^(24 + 7*n) + q^(27 + 7*n) + q^(23 + 8*n) + # f(n+1)
     2*q^(24 + 8*n) + 2*q^(25 + 8*n) + 2*q^(26 + 8*n) + 2*q^(27 + 8*n) + q^(28 + 8*n) - q^(30 + 8*n) - q^(31 + 8*n) - q^(27 + 9*n) - 
     q^(28 + 9*n) - q^(29 + 9*n) + q^(32 + 9*n) + q^(33 + 9*n) + q^(34 + 9*n) + q^(30 + 10*n) + q^(31 + 10*n) + q^(32 + 10*n) + 
     q^(33 + 10*n) + q^(34 + 10*n) + q^(35 + 10*n) + q^(35 + 11*n) + q^(36 + 11*n) + q^(37 + 11*n) + q^(38 + 11*n) + q^(39 + 11*n) + 
     q^(40 + 11*n) - q^(38 + 12*n) - q^(39 + 12*n) - q^(40 + 12*n) + q^(42 + 13*n) + q^(43 + 13*n) + q^(44 + 13*n) + q^(45 + 13*n) + 
     q^(46 + 13*n) + q^(47 + 13*n)), 
   (q^(12 + 3*n) + q^(13 + 3*n) - 2*q^(15 + 4*n) - 3*q^(16 + 4*n) - 2*q^(17 + 4*n) - q^(18 + 4*n) + q^(18 + 5*n) + 2*q^(19 + 5*n) + # f(n+2)
     2*q^(20 + 5*n) + q^(21 + 5*n) + q^(22 + 5*n) - q^(19 + 6*n) - q^(20 + 6*n) - 3*q^(21 + 6*n) - 3*q^(22 + 6*n) - 3*q^(23 + 6*n) - 
     q^(24 + 6*n) - q^(25 + 6*n) + q^(21 + 7*n) + 2*q^(22 + 7*n) + 3*q^(23 + 7*n) + 3*q^(24 + 7*n) + 3*q^(25 + 7*n) + q^(26 + 7*n) - 
     2*q^(28 + 7*n) - q^(29 + 7*n) - q^(30 + 7*n) - q^(24 + 8*n) - 2*q^(25 + 8*n) - 3*q^(26 + 8*n) - 2*q^(27 + 8*n) - 3*q^(28 + 8*n) - 
     q^(29 + 8*n) + 2*q^(31 + 8*n) + 2*q^(32 + 8*n) + q^(33 + 8*n) + q^(27 + 9*n) + 3*q^(28 + 9*n) + 4*q^(29 + 9*n) + 4*q^(30 + 9*n) + 
     2*q^(31 + 9*n) + q^(32 + 9*n) - 2*q^(33 + 9*n) - 2*q^(34 + 9*n) - 4*q^(35 + 9*n) - 2*q^(36 + 9*n) - q^(37 + 9*n) - q^(31 + 10*n) - 
     q^(32 + 10*n) - q^(33 + 10*n) + q^(34 + 10*n) + 2*q^(35 + 10*n) + 3*q^(36 + 10*n) + 3*q^(37 + 10*n) + 2*q^(38 + 10*n) + q^(39 + 10*n) + 
     q^(35 + 11*n) - 2*q^(38 + 11*n) - 2*q^(39 + 11*n) - 3*q^(40 + 11*n) - 2*q^(41 + 11*n) - q^(42 + 11*n) + q^(39 + 12*n) + 2*q^(40 + 12*n) + 
     3*q^(41 + 12*n) + 3*q^(42 + 12*n) + 2*q^(43 + 12*n) + q^(44 + 12*n) - q^(43 + 13*n) - q^(44 + 13*n) - q^(45 + 13*n) + q^(47 + 14*n) + 
     q^(48 + 14*n) + q^(49 + 14*n)), 
   (q^(10 + 2*n) + q^(12 + 3*n) - q^(12 + 4*n) - 2*q^(13 + 4*n) - 4*q^(14 + 4*n) - 5*q^(15 + 4*n) - 5*q^(16 + 4*n) - 2*q^(17 + 4*n) - # f(n+3)
     q^(18 + 4*n) + q^(15 + 5*n) + 2*q^(16 + 5*n) + 4*q^(17 + 5*n) + 5*q^(18 + 5*n) + 6*q^(19 + 5*n) + 5*q^(20 + 5*n) + 3*q^(21 + 5*n) + 
     q^(22 + 5*n) - q^(18 + 6*n) - 4*q^(19 + 6*n) - 6*q^(20 + 6*n) - 8*q^(21 + 6*n) - 7*q^(22 + 6*n) - 7*q^(23 + 6*n) - 4*q^(24 + 6*n) - 
     2*q^(25 + 6*n) + q^(22 + 7*n) + 2*q^(23 + 7*n) + 2*q^(24 + 7*n) + q^(25 + 7*n) - q^(28 + 7*n) + q^(25 + 8*n) + q^(26 + 8*n) + 
     2*q^(27 + 8*n) + 2*q^(28 + 8*n) + 3*q^(29 + 8*n) + 3*q^(30 + 8*n) + 3*q^(31 + 8*n) + 2*q^(32 + 8*n) - q^(28 + 9*n) - 3*q^(29 + 9*n) - 
     5*q^(30 + 9*n) - 7*q^(31 + 9*n) - 7*q^(32 + 9*n) - 7*q^(33 + 9*n) - 5*q^(34 + 9*n) - 4*q^(35 + 9*n) - q^(36 + 9*n) + q^(32 + 10*n) + 
     3*q^(33 + 10*n) + 5*q^(34 + 10*n) + 6*q^(35 + 10*n) + 4*q^(36 + 10*n) + 3*q^(37 + 10*n) + q^(38 + 10*n) + q^(39 + 10*n) - q^(36 + 11*n) - 
     2*q^(37 + 11*n) - 3*q^(38 + 11*n) - 2*q^(39 + 11*n) - q^(40 + 11*n) - q^(41 + 12*n) - q^(42 + 12*n) - 2*q^(43 + 12*n) - q^(44 + 12*n) - 
     q^(45 + 12*n) + q^(44 + 13*n) + 2*q^(45 + 13*n) + 2*q^(46 + 13*n) + q^(47 + 13*n) - q^(48 + 14*n) - q^(49 + 14*n) - q^(50 + 14*n)),
    (-1 + q^(2 + n) + 2*q^(3 + n) + 2*q^(4 + n) - q^(5 + 2*n) - 2*q^(6 + 2*n) - 3*q^(7 + 2*n) - q^(8 + 2*n) + q^(7 + 3*n) + q^(8 + 3*n) + # f(n+4)
     2*q^(9 + 3*n) + q^(10 + 3*n) + q^(11 + 3*n) - q^(13 + 5*n) - q^(14 + 5*n) - 2*q^(15 + 5*n) - q^(16 + 5*n) - q^(17 + 5*n) + q^(16 + 6*n) + 
     2*q^(17 + 6*n) + 2*q^(18 + 6*n) + q^(19 + 6*n) - q^(20 + 7*n) - q^(21 + 7*n) - q^(22 + 7*n))
]

In [17]:
def coeff_to_operator(coeff, dest_q, dest_Q):
    import re
    opers = coeff.factor().operands()
    if len(opers) == 2:
        big, small = opers
        extra = 1
    else:
        big, small, extra = opers
    string_big = str(big.simplify_full())
    string_small = str(small.simplify_full())
    regs = [(r"q\^\((\d*)\*n\)", r"Q^(\1)"),
            (r"q\^\((\d*)\*n \+ (\d*)\)", r"q^(\2)*Q^(\1)"),
            (r"q\^\(n \+ (\d*)\)", r"q^(\1)*Q^(1)"),
            (r"q\^n", r"Q^(1)"),
            (r"\^", r"**")]
    def apply_regs(regs, string):
        for reg in regs:
            string = re.sub(*reg, string)
        return string
    
    q = dest_q; Q = dest_Q
    return dest_Q.parent()(extra)*eval(apply_regs(regs, string_big)) * eval(apply_regs(regs, string_small))

In [15]:
coeffs = [
    -q^(3*n)*(1 + q^n)*(-q + q^n)*(-q^3 + q^(2*n))*(1 - q^n + q^(2*n)), # f(n-2)
    -q^(2 + 2*n)*(q^2 + q^(3*n) - q^(5*n) + q^(6*n) + q^(2 + n) + q^(3 + n) - q^(1 + 2*n) - q^(2 + 2*n) + q^(1 + 3*n) + q^(1 + 4*n) +  q^(2 + 4*n) - q^(1 + 5*n)), # f(n-1)
    q^2*(q^2 + q^(2*n) - q^(1 + n)) #f(n)
]

In [12]:
L

q^4 - q^3*Q + (-q^4+q^2)*Q^2 + (-q^5-q^4)*Q^3 + (q^4+q^3)*Q^4 + (-q^3-q^2)*Q^5 - q^5*Q^3*Si^2 + (-q^4-q^3)*Q^6 + q^5*Q^4*Si^2 + (q^3+q^2)*Q^7 + (-q^5+q^3+q^2)*Q^5*Si^2 - q^2*Q^8 + (-q^3-q^2)*Q^6*Si^2 + (q^3+q^2-1)*Q^7*Si^2 + Q^8*Si^2 - Q^9*Si^2

Second (and smaller example):

In [37]:
coeffs2 = [
    q^(3*n + 6)*(1 - q^(1 + n))*(q + q^(3*n + 6) - q^(3 + n)),
    q^(-3 + n)*(-q^7 + q^(8*n + 16) + q^(8 + n) + q^(9 + n) + q^(8 + 2*n) +
        q^(9 + 2*n) - q^(9 + 3*n) - 2*q^(10 + 3*n) - 2*q^(11 + 3*n) - 
        q^(12 + 3*n) + q^(11 + 4*n) + q^(12 + 4*n) + q^(13 + 4*n) + 
        q^(11 + 5*n) + q^(12 + 5*n) + q^(13 + 5*n) + q^(14 + 5*n) - 
        q^(13 + 6*n) - q^(14 + 6*n) - q^(15 + 6*n)),
    -(q^4 + q^(3*(n+2)) - q^(5 + n))    
]

In [38]:
LL = sum([coeff_to_operator(coeffs2[i], A(q), A(Q))*S^i for i in range(len(coeffs2))]);

ValueError: too many values to unpack (expected 3)

### Paper example

Here we are going to prove some identity (add reference) regarding the Rogers-Ramanujan $q$-series.

More precisely, we would like to get a solution to the following recurrence equation using some special basis:
$$L = -E^3 +(1 + q^6q_n^3)E^2 +(b q^5 q_n^3 + a q^7 q_n^3 + a b q^{12} q_n^6) E + (a b q^9 q_n^6 - a b q^{12} q_n^9),$$
where $a,b, q$ are transcendental elements and $q_n$ represents the multiplication by $q^n$ operator.

In [100]:
B = QBinomialBasis(1,e=3);
C.<a,b> = B.base[]
BwZ = B.change_base(C)
q = BwZ.base('q'); 
a = BwZ.base('a');
b = BwZ.base('b');

OE, (q_3n, E) = get_qshift_algebra("q_3n", "q", "E", power=3, rational=False, base=BwZ.base)
L = -E^3 +(1 + q^6*q_3n) *E^2 +(b * q^5 * q_3n + a *q^7 *q_3n + a *b *q^12 *q_3n^2) *E + (a *b *q^9 *q_3n^2 - a *b *q^12 *q_3n^3)
y = solution(L, [1, 1, 1+ b*q^2 + q^3 + a*q^4 + a * b * q^6], name_q = "q"); 
y[:5]

[1,
 1,
 (q^6)*a*b + (q^4)*a + (q^2)*b + (q^3 + 1),
 (q^12 + q^9 + q^6)*a*b + (q^10 + q^7 + q^4)*a + (q^8 + q^5 + q^2)*b + (q^9 + q^6 + q^3 + 1),
 (q^24)*a^2*b^2 + (q^22 + q^16)*a^2*b + (q^20 + q^14)*a*b^2 + (q^14)*a^2 + (q^21 + 2*q^18 + 2*q^15 + 3*q^12 + q^9 + q^6)*a*b + (q^10)*b^2 + (q^19 + q^16 + 2*q^13 + 2*q^10 + q^7 + q^4)*a + (q^17 + q^14 + 2*q^11 + 2*q^8 + q^5 + q^2)*b + (q^18 + q^15 + q^12 + 2*q^9 + q^6 + q^3 + 1)]

We want to compute a solution of the form
$$y(n) = \sum_{k\geq 0} c(k)q^{n^2-n - 6nk} \qbinomb{n}{2k}{q^3}.$$
For doing so, we need first to pull out the parts that only involve $n$ inside the sum, move it to the left, compute an annihilator for the new left-hand side of the equation and apply the compatibility theory to the product basis of $q^{-3nk}$ and $\qbinomb{n}{k}{q^3}$.

#### Finding the new recurrence

If we pull-out the terms in $n$ inside the sum (i.e., $q^{n^2-n}$) then we have
$$\tilde{y}(n) = y(n)q^{n-n^2} = \sum_{k\geq 0} c(k)q^{-6nk} \qbinomb{n}{2k}{q^3}.$$
Since $q^{n-n^2}$ is $q$-holonomic, the recursion for $\tilde{y}(n)$ can be found quite easily. however, since we want an operator compatible with both $q^{3n}$ and $\qbinomb{n}{k}{q^3}$, then we need to use only product of $q^{3n}$ in the recurrences:

In [98]:
## T annihilates $y^{n-n^2}$
OE2, (q_n, E2) = get_qshift_algebra("q_n", "q", "E", power=1, rational=False, base=BwZ.base)
T = q_n^2*E2 - 1

In [99]:
## By construction, the following operator also annihilates $y^{n-n^2}
(T^2 + 3*q_n^2*E2)*T

(q^6)*q_n^6*E^3 - 1

In [101]:
## We use that formula to build the operator in the same ring as $L$
nT = q^6*q_3n^2*E^3 - 1

In [107]:
## The following operator annihilates $y(n) q^{n-n^2}$
#L_tilde = L.symmetric_product(nT)
#y_tilde = solution(L_tilde, (y*b), name_q = "q"); y_tilde[:5]
## Currently not working due to ore_algebra issues. Either ask Manuel about it or use Mathematica

#### Computing the product basis

Now we are going to build the product basis between $q^{3kn}$ and $\qbinom{

### Paper example without Product Basis

In [2]:
B = QBinomialBasis(1,e=3);
C.<a,b> = B.base[]
BwZ = B.change_base(C)
q = BwZ.base('q'); 
a = BwZ.base('a');
b = BwZ.base('b');

OE, (q_3n, E) = get_qshift_algebra("q_3n", "q", "E", power=3, rational=False, base=BwZ.base)
L = -E^3 +(1 + q^6*q_3n) *E^2 +(b * q^5 * q_3n + a *q^7 *q_3n + a *b *q^12 *q_3n^2) *E + (a *b *q^9 *q_3n^2 - a *b *q^12 *q_3n^3)
y = solution(L, [1, 1, 1+ b*q^2 + q^3 + a*q^4 + a * b * q^6], name_q = "q"); 
y[:5]

[1,
 1,
 (q^6)*a*b + (q^4)*a + (q^2)*b + (q^3 + 1),
 (q^12 + q^9 + q^6)*a*b + (q^10 + q^7 + q^4)*a + (q^8 + q^5 + q^2)*b + (q^9 + q^6 + q^3 + 1),
 (q^24)*a^2*b^2 + (q^22 + q^16)*a^2*b + (q^20 + q^14)*a*b^2 + (q^14)*a^2 + (q^21 + 2*q^18 + 2*q^15 + 3*q^12 + q^9 + q^6)*a*b + (q^10)*b^2 + (q^19 + q^16 + 2*q^13 + 2*q^10 + q^7 + q^4)*a + (q^17 + q^14 + 2*q^11 + 2*q^8 + q^5 + q^2)*b + (q^18 + q^15 + q^12 + 2*q^9 + q^6 + q^3 + 1)]

In [3]:
L_comp = BwZ.recurrence(L,sections=2, output="ore")

In [4]:
f_L = L_comp[(0,0)].gcrd(L_comp[(1,0)]);f_L

Sk^3 + (((-q^30)*a*b + (-q^27))*q_k^12 + ((-q^16)*a + (-q^14)*b)*q_k^6)*Sk^2 + ((q^45)*a*b*q_k^24 + ((-q^36 - q^33)*a*b)*q_k^18 + (q^24)*a*b*q_k^12)*Sk

In [6]:
c = solution(f_L, BwZ.inner_init_values(y, f_L.order(), f_L, section=2, shift=0))

In [7]:
c[4].factor()

((q^44)) * ((q^52)*a^4*b^4 + (q^50 + q^44 + q^38 + q^32)*a^4*b^3 + (q^48 + q^42 + q^36 + q^30)*a^3*b^4 + (q^42 + q^36 + 2*q^30 + q^24 + q^18)*a^4*b^2 + (q^49 + q^46 + q^43 + 2*q^40 + q^37 + 3*q^34 + q^31 + 3*q^28 + 2*q^22 + q^16)*a^3*b^3 + (q^38 + q^32 + 2*q^26 + q^20 + q^14)*a^2*b^4 + (q^28 + q^22 + q^16 + q^10)*a^4*b + (q^47 + 2*q^41 + q^38 + 3*q^35 + 2*q^32 + 3*q^29 + 3*q^26 + 2*q^23 + 3*q^20 + q^17 + 2*q^14 + q^8)*a^3*b^2 + (q^45 + 2*q^39 + q^36 + 3*q^33 + 2*q^30 + 3*q^27 + 3*q^24 + 2*q^21 + 3*q^18 + q^15 + 2*q^12 + q^6)*a^2*b^3 + (q^22 + q^16 + q^10 + q^4)*a*b^4 + (q^8)*a^4 + (q^39 + 2*q^33 + 3*q^27 + q^24 + 3*q^21 + q^18 + 2*q^15 + q^12 + q^9 + q^6)*a^3*b + (q^46 + q^43 + q^40 + 3*q^37 + 2*q^34 + 5*q^31 + 2*q^28 + 6*q^25 + 2*q^22 + 5*q^19 + 2*q^16 + 3*q^13 + q^10 + q^7 + q^4)*a^2*b^2 + (q^35 + 2*q^29 + 3*q^23 + q^20 + 3*q^17 + q^14 + 2*q^11 + q^8 + q^5 + q^2)*a*b^3 + b^4 + (q^25 + q^19 + q^13 + q^7)*a^3 + (q^44 + 2*q^38 + q^35 + 3*q^32 + 2*q^29 + 3*q^26 + 3*q^23 + 2*q^20 + 3*q^17 + q^14 + 2*q^11 + q^5)*a^2*b + (q^42 + 2*q^36 + q^33 + 3*q^30 + 2*q^27 + 3*q^24 + 3*q^21 + 2*q^18 + 3*q^15 + q^12 + 2*q^9 + q^3)*a*b^2 + (q^19 + q^13 + q^7 + q)*b^3 + (q^36 + q^30 + 2*q^24 + q^18 + q^12)*a^2 + (q^43 + q^40 + q^37 + 2*q^34 + q^31 + 3*q^28 + q^25 + 3*q^22 + 2*q^16 + q^10)*a*b + (q^32 + q^26 + 2*q^20 + q^14 + q^8)*b^2 + (q^41 + q^35 + q^29 + q^23)*a + (q^39 + q^33 + q^27 + q^21)*b + (q^40))

In [8]:
nY = Sequence(lambda n : sum(c[k]*BwZ[2*k][n] for k in range(n+1)), universe=y.universe)

In [9]:
y[:3]

[1, 1, (q^6)*a*b + (q^4)*a + (q^2)*b + (q^3 + 1)]

In [11]:
(nY - y)[:15]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## **Symmetric Q-Binomials** (<span style="color:green">TODO: Review this section</span>)

We want to study the different type of $q$-binomial basis like:
$$\qbinom{2n}{n+k}$$

In [3]:
B = LambdaSequence(lambda k,n : gaussian_binomial(n, k, q), q.parent(), 2)
B2 = LambdaSequence(lambda k,n : gaussian_binomial(2*n, k, q), q.parent(), 2)
SB = LambdaSequence(lambda k,n : gaussian_binomial(2*n, n+k, q), q.parent(), 2)
Qn = LambdaSequence(lambda k,n : q**n, q.parent(), 2)
Qk = LambdaSequence(lambda k,n : q**k, q.parent(), 2)
Qnk = LambdaSequence(lambda k,n : q**(n+k), q.parent(), 2)

Getting the recurrence (not full comaptibility) of the symmetric $q$-binomial with the **multiplication by $q^{2n}$**. This is obtained by applying a substitution $K = n+k$, applying knowledge on $\binom{2n}{K}$ and rolling back the changes:
$$q^{2n}\qbinom{2n}{n+k} = q^{n+k}(q^{n+k+1} - 1) \qbinom{2n}{n+k+1} + q^{n+k}\qbinom{2n}{n+k}$$
$$q^{2n}\qbinom{2n}{K} = q^{K}(q^{K+1} - 1) \qbinom{2n}{K+1} + q^{K}\qbinom{2n}{K}$$

$$a_n = \sum_k c_{n-K}\qbinom{2n}{K}$$

$$S(n) = n, S(K) = k+1$$

In [4]:
get_matrix(Qn*Qn*SB - Qn*Qk*(q*Qn*Qk - 1)*SB.shift(1,0) - Qn*Qk*SB)

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

From this formula we can also obtain an identity for the multiplication by $q^n$:
$$q^{n}\qbinom{2n}{n+k} = q^{k}(q^{n+k+1} - 1) \qbinom{2n}{n+k+1} + q^{k}\qbinom{2n}{n+k}$$

In [5]:
get_matrix(Qn*SB - Qk*(q*Qn*Qk - 1)*SB.shift(1,0) - Qk*SB)

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

Finally, we can obtain the same type of identity for the **shift w.r.t. $n$**:
$$\qbinom{2n+2}{n+k+1} = q^{2(n+k+1)}\qbinom{2n}{n+k+1} + q^{n+k}(1+q)\qbinom{2n}{n+k} +\qbinom{2n}{n+k-1}$$

In [6]:
get_matrix(SB.shift(0,1) - Qn*Qn*Qk*Qk*q^2*SB.shift(1,0) - Qn*Qk*(1+q)*SB - SB.shift(-1,0))

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

All these identities have the same flaw: they make use of $q^n$. In particular, the first and the third can be written in terms of $q^{n+k}$. Maybe there is something to graps there.

### Trying to exploit the recursion on the shift in $n$

When we consider the shift w.r.t. $n$, we used the usual recurrence to reduce the top argument of the $q$-binomial. However, there is a symmetric identity in place here:
$$\qbinom{m}{r} = q^m\qbinom{m-1}{r} + \qbinom{m-1}{r-1} = \qbinom{m-1}{r} + q^{m-r}\qbinom{m-1}{r-1}.$$

In [17]:
# Second identity
get_matrix(B.shift(0,1) - B - q*Qn/Qk*B.shift(-1,0))

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

In general, we were not using the secon identity because it mixed up the upper and lower arguments of the $q$-binomial. But now we already have them mixed up anyway. Hence, it is worth to see how we can use both identities. This will lead up to a total of 8 options:

* [1] -> (1,1), (1,2), (2,1), (2,2)
* [2] -> (1,1), (1,2), (2,1), (2,2)

Let us check how the final compatibility equation remains and if we can extract some information from them:

#### Starting with first identity

$$\binom{2n+2}{n+k+1} = q^{n+k+1}\qbinom{2n+1}{n+k+1} + \qbinom{2n+1}{n+k}.$$

Now for each of the involved binomials, we have two options to reduce the upper argumetn one more time, to fall again into our basis:

* (1,1):
$$\qbinom{2n+2}{n+k+1} = q^{2(n+k+1)}\qbinom{2n}{n+k+1} + q^{n+k}(q + 1)\qbinom{2n}{n+k} + \qbinom{2n}{n+k-1}.$$

In [9]:
get_matrix(SB.shift(0,1) - q^2*Qn*Qn*Qk*Qk*SB.shift(1,0) - Qn*Qk*(1+q)*SB - SB.shift(-1,0))

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

* (1,2):
$$\binom{2n+2}{n+k+1} = q^{2(n+k+1)}\qbinom{2n}{n+k+1} + (q^{n+k+1} + 1)\qbinom{2n}{n+k} + q^{n-k+1}\qbinom{2n}{n+k-1}.$$

In [18]:
get_matrix(SB.shift(0,1) - q^2*Qn*Qn*Qk*Qk*SB.shift(1,0) - (Qn*Qk*q + 1)*SB - Qn/Qk*q*SB.shift(-1,0))

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

* (2,1):
$$\binom{2n+2}{n+k+1} = q^{n+k+1}\qbinom{2n}{n+k+1} + q^n(q^{n+1} + q^k)\qbinom{2n}{n+k} + \qbinom{2n}{n+k-1}.$$

In [21]:
get_matrix(SB.shift(0,1) - Qn*Qk*q*SB.shift(1,0) - Qn*(Qn*q + Qk)*SB - SB.shift(-1,0))

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

* (2,2):
$$\binom{2n+2}{n+k+1} = q^{n+k+1}\qbinom{2n}{n+k+1} + (q^{2n+1}+1)\qbinom{2n}{n+k} + q^{n-k+1}\qbinom{2n}{n+k-1}.$$

In [22]:
get_matrix(SB.shift(0,1) - Qn*Qk*q*SB.shift(1,0) - (Qn*Qn*q + 1)*SB - Qn/Qk*q*SB.shift(-1,0))

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

#### Starting with second identity

$$\binom{2n+2}{n+k+1} = \qbinom{2n+1}{n+k+1} + q^{n-k+1}\qbinom{2n+1}{n+k}.$$

Now for each of the involved binomials, we have two options to reduce the upper argumetn one more time, to fall again into our basis:

* (1,1) --> [1] (2,2):
$$\binom{2n+2}{n+k+1} = q^{n+k+1}\qbinom{2n}{n+k+1} + (q^{2n+1} + 1)\qbinom{2n}{n+k} + q^{n-k+1}\qbinom{2n}{n+k-1}.$$
* (1,2):
$$\binom{2n+2}{n+k+1} = q^{n+k+1}\qbinom{2n}{n+k+1} + (q^{n-k+1} + 1)\qbinom{2n}{n+k} + q^{2n-2k+2}\qbinom{2n}{n+k-1}).$$

In [24]:
get_matrix(SB.shift(0,1) - Qn*Qk*q*SB.shift(1,0) - (Qn/Qk*q +1)*SB - Qn*Qn/Qk/Qk*q^2 * SB.shift(-1,0))

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

* (2,1):
$$\binom{2n+2}{n+k+1} = \qbinom{2n}{n+k+1} + q^n(q^{n+1} + q^{-k})\qbinom{2n}{n+k} + q^{n-k+1}\qbinom{2n}{n+k-1}.$$

In [25]:
get_matrix(SB.shift(0,1) - SB.shift(1,0) - Qn*(Qn*q + 1/Qk)*SB - Qn/Qk*q * SB.shift(-1,0))

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

* (2,2):
$$\binom{2n+2}{n+k+1} = \qbinom{2n}{n+k+1} + q^{n-k}(1 + q)\qbinom{2n}{n+k} + q^{2n-2k+2}\qbinom{2n}{n+k-1}.$$

In [27]:
get_matrix(SB.shift(0,1) - SB.shift(1,0) - Qn/Qk*(1+q)*SB - Qn*Qn/Qk/Qk*q^2*SB.shift(-1,0))

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

#### Conclusion

We obtained a total of 7 different mixed compatibilities (involving both $q^n$ and $q^k$) for the shift in $n$. Hence, every difference is zero and this may provide, if manipulated, a compatibility for the multiplication by $q^n$?

## **New example (30/06/2023)** (<span style="color:green">TODO: review this section</span>)

We are going to try and prove the following identity:
$$a_n(x) = \left(\frac{-q}{x};q^2\right)_n \left(-xq;q^2\right) = \sum_j q^{2j}x^j\qbinomb{2n}{n+j}{q^2},$$
after a small run on the HolonomicFunctions softare in Mathematica, we arrived that the left-hand side satisfies the following recurrence:

$$a_{n+1}(x) = \frac{q^{2n+1}(1+q^{-2n-1}x)(1+q^{2n+1}x)}{x}a_{n}(x)\text{, with } a_0(x) = 1$$

### Trying the direct basis (<span style="color:green">TODO: review this section</span>)

Let us consider the sequence $B_k(n) = q^{n^2-2nk}\qbinomb{2n}{k}{q^2}$, and write $a_n = \sum_k c_k B_k(n)$. Can we find a $k$-recurrence for $c_k$?

In [3]:

B = LambdaSequence(lambda k,n : q**(n**2-2*n*k)*gaussian_binomial(2*n,k,q^2), F, 2)
get_matrix(B)

[                                                                                                                                             1                                                                                                                                              q                                                                                                                                            q^4                                                                                                                                            q^9                                                                                                                                           q^16]
[                                                                                                                                             0                                                                                                                                    (q^2 + 1)/q                                                                                                                            q^6 + q^4 + q^2 + 1                                                                                                            q^13 + q^11 + q^9 + q^7 + q^5 + q^3                                                                                           q^22 + q^20 + q^18 + q^16 + q^14 + q^12 + q^10 + q^8]
[                                                                                                                                             0                                                                                                                                          1/q^3                                                                                                              (q^8 + q^6 + 2*q^4 + q^2 + 1)/q^4                                                                          (q^16 + q^14 + 2*q^12 + 2*q^10 + 3*q^8 + 2*q^6 + 2*q^4 + q^2 + 1)/q^3                                            q^24 + q^22 + 2*q^20 + 2*q^18 + 3*q^16 + 3*q^14 + 4*q^12 + 3*q^10 + 3*q^8 + 2*q^6 + 2*q^4 + q^2 + 1]
[                                                                                                                                             0                                                                                                                                              0                                                                                                                      (q^6 + q^4 + q^2 + 1)/q^8                                                                 (q^18 + q^16 + 2*q^14 + 3*q^12 + 3*q^10 + 3*q^8 + 3*q^6 + 2*q^4 + q^2 + 1)/q^9           (q^30 + q^28 + 2*q^26 + 3*q^24 + 4*q^22 + 5*q^20 + 6*q^18 + 6*q^16 + 6*q^14 + 6*q^12 + 5*q^10 + 4*q^8 + 3*q^6 + 2*q^4 + q^2 + 1)/q^8]
[                                                                                                                                             0                                                                                                                                              0                                                                                                                                         1/q^12                                                                         (q^16 + q^14 + 2*q^12 + 2*q^10 + 3*q^8 + 2*q^6 + 2*q^4 + q^2 + 1)/q^15 (q^32 + q^30 + 2*q^28 + 3*q^26 + 5*q^24 + 5*q^22 + 7*q^20 + 7*q^18 + 8*q^16 + 7*q^14 + 7*q^12 + 5*q^10 + 5*q^8 + 3*q^6 + 2*q^4 + q^2 + 1)/q^16]

In order to compute the recurrence for the $c_k$, we are going to need the compatiblity of $B_k(n)$ with respect to the shift in $n$ and the multiplication by $q^{2n}$ (maybe we need to the compatibility with $q^{-2n}$). 

#### Compatibility with $n\mapsto n+1$

$$\begin{array}{rl}
    B_k(n+1) & = q^{(n+1)^2-2(n+1)k}\qbinomb{2n+2}{k}{q^2} = q^{n^2+2n+1-2nk-k}\qbinomb{2n+2}{k}{q^2} = q^{n^2+2n+1-2nk-k}\left(q^{4n+4}\qbinomb{2n+1}{k}{q^2} + \qbinomb{2n+1}{k-1}{q^2}\right)\\
             & = q^{n^2+2n+1-2nk-k}\left(q^{8n+6}\qbinomb{2n}{k}{q^2} + q^{4n+2}(1+q^2)\qbinomb{2n}{k-1}{q^2} + \qbinomb{2n}{k-2}{q^2}\right) \\
             & = q^{10n - k +7}q^{n^2 - 2nk}\qbinomb{2n}{k}{q^2} + q^{8n+3-k}(1+q^2)q^{n^2-2nk-2n}\qbinomb{2n}{k-1}{q^2} + q^{6n+1-k}q^{n^2 - 2nk -4n}\qbinomb{2n}{k-2}{q^2}\\
             & = q^{6n+1}\left(q^{4n - k + 6}B_k(n) + q^{2n - k + 2}B_{k-1}(n) + q^{-k} B_{k-2}(n)\right)
\end{array}$$

We need that $q^{2n}$ is compatible with $B_k(n)$.

#### Compatibility with $q^{2n}$

For this we can use the classical compatibility with the $q$-binomial of the multiplication by $q^n$.
$$\begin{array}{rl}
    q^{2n}B_k(n) & = q^{n^2-2nk}q^{2n}\qbinomb{2n}{k}{q^2} = q^{n^2-2nk-2n}\left(\frac{1}{a_{k+1}}\qbinomb{2n}{k+1}{q^2} - \frac{b_{k+1}}{a_{k+1}}\qbinomb{2n}{k}{q^2}\right)\\
                 & = \frac{q^{n^2-2n(k+1)}}{a_{k+1}}\qbinomb{2n}{k+1}{q^2} - q^{-2n}\frac{b_{k+1}}{a_{k+1}}q^{n^2-2nk}\qbinomb{2n}{k}{q^2} \\
                 & = \frac{1}{a_{k+1}}B_{k+1}(n) - q^{-2n}\frac{b_{k+1}}{a_{k+1}}B_k(n),
\end{array}$$

where the $a_k$ and $b_k$ are like in the binomial case:
$$a_k = \frac{q}{q^k(q^{k} - 1)},\qquad b_k = \frac{1}{1-q^{k}}.$$

Funny enough... It seems we need the compatibility with $q^{-2n}$.

#### Compatibility with $q^{-2n}$

$$q^{-2n}B_k(n) = q^{n^2 - 2n(k+1)}\qbinomb{2n}{k}{q^2}$$

### Changing the base of the $q$-Binomial (<span style="color:green">TODO: review this section</span>)

Let us now consider the basis $B_k(n) = \qbinom{2n}{k}$. Let us check the corresponding compatibilities of this basis:

In [40]:
B = QBinomialBasis(2)
B = B.change_base(B.base.base().extend_variables('x').fraction_field())
x = B.base("x"); q = B.q
a = Q_pochhammer(-q*x, q^2) * Q_pochhammer(-q/x, q^2)
get_matrix(B)

[QBinomialBasis] Creating the basis with a=2, c=0, t=0, e=1


[                                                                                                                               1                                                                                                                                1                                                                                                                                1                                                                                                                                1                                                                                                                                1]
[                                                                                                                               0                                                                                                                            q + 1                                                                                                                q^3 + q^2 + q + 1                                                                                                    q^5 + q^4 + q^3 + q^2 + q + 1                                                                                        q^7 + q^6 + q^5 + q^4 + q^3 + q^2 + q + 1]
[                                                                                                                               0                                                                                                                                1                                                                                                        q^4 + q^3 + 2*q^2 + q + 1                                                                        q^8 + q^7 + 2*q^6 + 2*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1                                     q^12 + q^11 + 2*q^10 + 2*q^9 + 3*q^8 + 3*q^7 + 4*q^6 + 3*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1]
[                                                                                                                               0                                                                                                                                0                                                                                                                q^3 + q^2 + q + 1                                                                q^9 + q^8 + 2*q^7 + 3*q^6 + 3*q^5 + 3*q^4 + 3*q^3 + 2*q^2 + q + 1          q^15 + q^14 + 2*q^13 + 3*q^12 + 4*q^11 + 5*q^10 + 6*q^9 + 6*q^8 + 6*q^7 + 6*q^6 + 5*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q + 1]
[                                                                                                                               0                                                                                                                                0                                                                                                                                1                                                                        q^8 + q^7 + 2*q^6 + 2*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1 q^16 + q^15 + 2*q^14 + 3*q^13 + 5*q^12 + 5*q^11 + 7*q^10 + 7*q^9 + 8*q^8 + 7*q^7 + 7*q^6 + 5*q^5 + 5*q^4 + 3*q^3 + 2*q^2 + q + 1]

In [42]:
a[:4 ]

[1,
 (q*x^2 + (q^2 + 1)*x + q)/x,
 (q^4*x^4 + (q^7 + q^5 + q^3 + q)*x^3 + (q^8 + q^6 + 2*q^4 + q^2 + 1)*x^2 + (q^7 + q^5 + q^3 + q)*x + q^4)/x^2,
 (q^9*x^6 + (q^14 + q^12 + q^10 + q^8 + q^6 + q^4)*x^5 + (q^17 + q^15 + 2*q^13 + 2*q^11 + 3*q^9 + 2*q^7 + 2*q^5 + q^3 + q)*x^4 + (q^18 + q^16 + 2*q^14 + 3*q^12 + 3*q^10 + 3*q^8 + 3*q^6 + 2*q^4 + q^2 + 1)*x^3 + (q^17 + q^15 + 2*q^13 + 2*q^11 + 3*q^9 + 2*q^7 + 2*q^5 + q^3 + q)*x^2 + (q^14 + q^12 + q^10 + q^8 + q^6 + q^4)*x + q^9)/x^3]

In [95]:
## Checking the recurrence of "a" --> "x*E - (q*q_n**2 + x)*(1 + x*q*q_n**2)"
(x*a.shift() - (x+Qn**2*q)*(1+Qn**2*q*x) * a)[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#### Compatibility with $n\mapsto n+1$

$$\begin{array}{rl}
    B_k(n+1) & = \qbinom{2n+2}{k} = q^k\qbinom{2n+1}{k} + \qbinom{2n+1}{k-1} \\
             & = q^{2k}\qbinom{2n}{k} + q^{k-1}(1+q)\qbinom{2n}{k-1} + \qbinom{2n}{k-2}
\end{array}$$

In [96]:
B.compatibility("E")

Compatibility condition (2, 0, 1) with following coefficient matrix:
[              1 ((q + 1)/q)*q_k           q_k^2]

Which translates into the recurrence:
$$\mathcal{R}(E_n) = S_k^2 + q^{k}(1+q)S_k + q^{2k}$$

In [97]:
B.recurrence("E")

Sk^2 + ((q + 1)*q_k)*Sk + q_k^2

#### Compatibility with $q^{2n}$

Using the compatibility with $q^n$ of $\qbinom{n}{k}$, we know that, for the sequences
$$a_k = \frac{q}{q^k(q^{k} - 1)},\qquad b_k = \frac{1}{1-q^{k}},$$
the following recurrence holds:
$$q^{2n}\qbinom{2n}{q} = \frac{1}{a_{k+1}}\qbinom{2n}{k+1} - \frac{b_{k+1}}{a_{k+1}}\qbinom{2n}{k}.$$

In [98]:
B.compatibility("q_2n")

Compatibility condition (0, 1, 1) with following coefficient matrix:
[          q_k q*q_k^2 - q_k]

This translates into the following recurrence:
$$\mathcal{R}(q^{2n}) = \frac{1}{a_k}S_k^{-1} - \frac{b_{k+1}}{a_{k+1}}$$

In [99]:
B.recurrence("q_2n")

(1/q*q_k^2 - 1/q*q_k)*Ski + q_k

#### Computing the final recurrence

We were starting from the following recurrence:
$$A_n(x) \cdot a_n(x) = xa_{n+1}(x) - q^{2n+1}(1+q^{-2n-1}x)(1+q^{2n+1}x)a_{n}(x) = 0$$

If we now write $a_n(x) = \sum_{k\geq 0} c_k(x)B_k(n)$, we could get a recurrence for the $c_k(x)$ as follows:
$$\cR(A_n(x)) = x\cR(E_n) - (q\cR(q^{2n})+x)(q\cR(q^{2n})+xq^2\cR(q^{2n})^2))$$

In [102]:
# Evaluating the recurrence
R_A = B.recurrence("x*E - (q*q_2n + x)*(1 + x*q*q_2n)")
show(R_A)
R_A.polynomial().monomials()

x*Sk^2 + (-1/q^2*x*q_k^4 + (((q + 1)/q^2)*x)*q_k^3 - 1/q*x*q_k^2)*Ski^2 + (((q + 1)*x)*q_k)*Sk + (((-q - 1)*x)*q_k^3 + (-x^2 + (q + 1)*x - 1)*q_k^2 + (x^2 + 1)*q_k)*Ski + ((-q^2 + 1)*x)*q_k^2 + (-q*x^2 - q)*q_k - x

[Sk^2, Ski^2, Sk, Ski, 1]

#### Computing the solution (<span style="color:green">TODO:Review this section</span>)

In order to obtain the solution, we would need now some initial conditions of the new sequence $c_k(x)$ that is defined in with the previous recurrence. We need the values for $c_0,c_1,c_2,c_3$ and $c_4$:

$$a_0(x) = \sum_{k \geq 0} c_k \qbinom{0}{k} = c_0(x),$$
$$a_1(x) = \sum_{k \geq 0} c_k \qbinom{2}{k} = c_0(x) + (q+1)c_1(x) + c_2(x)$$
$$a_2(x) = \sum_{k \geq 0} c_k \qbinom{4}{k} = c_0(x) + (q^3+q^2+q+1)c_1(x) + (q^4+q^3+2q^2+q+1)c_2(x) + (q^3+q^2+q+1)c_3(x) + c_4(x)$$

We can appreciate that there are many options for the initial values of $c_k(x)$. The only completely fixed is $c_0(x) = a_0(x)$. The others have some freedom.

* Any solution satisfying the constrains is a good solution for us? It looks like so.

Let us consider the solution with $c_0(x) = a_0(x)$, $c_1(x) = 1$ and $c_3(x) = 1$. Then, we must have:
$$c_2(x) = a_1(x) - a_0(x) - (q+1),$$
$$c_4(x) = a_2(x) - a_0(x) - (q^3+q^2+q+1) - (q^4+q^3+2q^2+q+1)(a_1(x) - a_0(x) - (q+1)) - (q^3+q^2+q+1).$$

In our case, we have:

In [180]:
c = []
c.append(an[0]) # adding c_0(x)
c.append(1) # adding a fixed value for c_1(x)
c.append(an[1] - B(0,1)*c[0] - B(1,1)*c[1]) # adding value for c_2(x)
c.append(1) # adding a fixed value for c_3(x)
c.append(an[2] - B(0,2)*c[0] - B(1,2)*c[1] - B(2,2)*c[2] - B(3,2)+c[3])

In [185]:
full_c = solution(FR_A, c)

### Using the standard $q$-binomial (<span style="color:green">TODO: review this section</span>)

We can also use the classical $q$-binomial basis that will always lead to a solution:

In [103]:
B = QBinomialBasis()
B = B.change_base(B.base.base().extend_variables('x').fraction_field())
x = B.base("x"); q=B.q
a = Q_pochhammer(-q*x, q^2) * Q_pochhammer(-q/x, q^2)
get_matrix(B)

[                        1                         1                         1                         1                         1]
[                        0                         1                     q + 1               q^2 + q + 1         q^3 + q^2 + q + 1]
[                        0                         0                         1               q^2 + q + 1 q^4 + q^3 + 2*q^2 + q + 1]
[                        0                         0                         0                         1         q^3 + q^2 + q + 1]
[                        0                         0                         0                         0                         1]

In [105]:
# Evaluating the recurrence
R_A = B.recurrence("x*E - (q*q_n**2 + x)*(1 + x*q*q_n**2)")
show(R_A)
R_A.polynomial().monomials()

(-1/q^14*x*q_k^8 + (((q^3 + q^2 + q + 1)/q^14)*x)*q_k^7 + (((-q^4 - q^3 - 2*q^2 - q - 1)/q^13)*x)*q_k^6 + (((q^3 + q^2 + q + 1)/q^11)*x)*q_k^5 - 1/q^8*x*q_k^4)*Ski^4 + ((((-q^3 - q^2 - q - 1)/q^10)*x)*q_k^7 + (((q^5 + 2*q^4 + 3*q^3 + 3*q^2 + 2*q + 1)/q^10)*x)*q_k^6 + (((-q^5 - 2*q^4 - 3*q^3 - 3*q^2 - 2*q - 1)/q^9)*x)*q_k^5 + (((q^3 + q^2 + q + 1)/q^7)*x)*q_k^4)*Ski^3 + ((((-q^4 - q^3 - 2*q^2 - q - 1)/q^6)*x)*q_k^6 + (((q^5 + 2*q^4 + 3*q^3 + 3*q^2 + 2*q + 1)/q^6)*x)*q_k^5 + (-1/q^3*x^2 + ((-q^4 - q^3 - 2*q^2 - q - 1)/q^5)*x - 1/q^3)*q_k^4 + (((q + 1)/q^3)*x^2 + (q + 1)/q^3)*q_k^3 + (-1/q^2*x^2 - 1/q^2)*q_k^2)*Ski^2 + x*Sk + ((((-q^3 - q^2 - q - 1)/q^2)*x)*q_k^5 + (((q^3 + q^2 + q + 1)/q^2)*x)*q_k^4 + (((-q - 1)/q)*x^2 + (-q - 1)/q)*q_k^3 + (((q + 1)/q)*x^2 + (q + 1)/q)*q_k^2)*Ski - q^2*x*q_k^4 + (-q*x^2 - q)*q_k^2 + x*q_k - x

[Ski^4, Ski^3, Ski^2, Sk, Ski, 1]

#### Computing the actual solution

Similar to what we tried before, we can see which initial values we need in order to get the final solution. In this case is much simpler, since the basis is triangular.

In [106]:
n = required_init(R_A)
init = get_matrix(B, n).solve_left(vector(a(i) for i in range(n)))
c = solution(R_A, init)

In [111]:
## Sanity check: the (c_k) obtain is correct:
a2 = Sequence(lambda n : sum(c(k)*B[k](n) for k in range(2*n+1)), universe=B.base, dim=1)
(a2-a).almost_zero(10)

True

#### Repeating the process

We can, now that we obtained the $(c_k)$, we can also express it as:
$$c_k = \sum_{l\geq 0} d_l \qbinom{k}{l},$$

In [119]:
ann_c = B.recurrence("x*E - (q*q_n**2 + x)*(1 + x*q*q_n**2)", output="ore")
ann_d = B.recurrence(str(ann_c).replace("q_k", "q_n").replace("Sk", "E"))
ann_d.polynomial().monomials()

[Ski^8,
 Ski^7,
 Ski^6,
 Sk^5,
 Ski^5,
 Sk^4,
 Ski^4,
 Sk^3,
 Ski^3,
 Sk^2,
 Ski^2,
 Sk,
 Ski,
 1]

In [120]:
n = required_init(ann_d)
init_d = get_matrix(B, n).solve_left(vector(c(i) for i in range(n)))
d = solution(ann_d, init_d)

In [122]:
## Sanity check: the (c_l) obtain is correct:
c2 = Sequence(lambda n : sum(d(k)*B[k](n) for k in range(2*n+1)), universe=B.base, dim=1)
(c2-c).almost_zero(10)

True

## **Building generic $q$-binomial basis** (<span style="color:green">TODO: review this section</span>)

Let us build the following type of $q$-binomial basis:
$$\left\{\qbinomb{an}{k}{q^b}\right\}_{k\in\mathbb{N}},\ \text{where}\ a,b\in \mathbb{N}.$$

The thing is, as we have checked in other ocasion, that these basss will be compatible with the shift in $n$ ($E: n \mapsto (n+1)$) and the multiplication by $q^{abn}$. In this section we are going to build the code and the compatibilities generically so they can be used later for particular examples.

### * Compatibility with the shift $E$

If we simply compute the shift w.r.t. $n$, we see that the top argument is shifted by $a$, hence, using the usual $q$-binomial identity $a$ times, we will bring back the top argument to $an$ but changing the lower argument with shifts in $k$. Namely:
$$\qbinomb{a(n+1)}{k}{q^b} = q^{bk}\qbinomb{an + a - 1}{k}{q^b} + \qbinomb{an+a-1}{k-1}{q^b}.$$

If we repeat one more time, we will get:
$$\qbinomb{a(n+1)}{k}{q^b} = q^{2bk}\qbinomb{an + a - 2}{k}{q^b} + q^{b(k-1)}(q^b + 1)\qbinomb{an + a - 1}{k-1}{q^b} + \qbinomb{an+a-2}{k-2}{q^b}.$$

Iterating this formula $a$ times, we get something of the form:
$$\qbinomb{a(n+1)}{k}{q^b} = \sum_{i=0}^a c_{i}(k)\qbinomb{an}{k-i}{q^b}.$$

These $c_i(k)$ will be polynomials in $q^{bk}$. They can be restated as polynomials in $q^k$, so the usual recurrences will be created in the end. However, we would need to have a fixed $b$, so no parametric computation can be done w.r.t. $b$.

### * Compatibility with the multiplication by $q^{abn}$

For the compatibility with $q^{ab}$ we just need to look to the usual $q$-factorial formula for the $q$-binomial:
$$\qbinom{n}{k+1} = (\alpha(q^k)q^n + \beta(q^k))\qbinom{n}{k}.$$

If we simply perform two substitutions of $n\mapsto an$ and $q \mapsto q^b$, then we get:
$$\qbinomb{an}{k+1}{q^b} = (\alpha(q^{bk}) q^{abn} + \beta(q^{bk}))\qbinomb{an}{k}{q^b}.$$

This will lead to a compatibility condition where the coefficients are rational functions in $q^{bk}$. Meaning that we can express the recurrences it in terms of the usual operator $q^k$.

### * **Quick note**

Since both compatibilities can be expressed as rational sequences in $q^{bk}$ we can either use this operator as an output or use $(q^k)^b$. The first case may allow us to have $b$ generic in some cases but may not fit properly in the usual setting, while the second case will have bigger coefficients.

In [16]:
B = QBinomialBasis(2,2)
B = B.change_base(B.base.base().extend_variables('x').fraction_field())

In [28]:
(B[1] - (q^2 + 1)*B[2] )[:10]

[0,
 0,
 -q^10 - 2*q^8 - 2*q^6 - 2*q^4 - q^2,
 -q^18 - 2*q^16 - 3*q^14 - 4*q^12 - 4*q^10 - 4*q^8 - 3*q^6 - 2*q^4 - q^2,
 -q^26 - 2*q^24 - 3*q^22 - 4*q^20 - 5*q^18 - 6*q^16 - 6*q^14 - 6*q^12 - 5*q^10 - 4*q^8 - 3*q^6 - 2*q^4 - q^2,
 -q^34 - 2*q^32 - 3*q^30 - 4*q^28 - 5*q^26 - 6*q^24 - 7*q^22 - 8*q^20 - 8*q^18 - 8*q^16 - 7*q^14 - 6*q^12 - 5*q^10 - 4*q^8 - 3*q^6 - 2*q^4 - q^2,
 -q^42 - 2*q^40 - 3*q^38 - 4*q^36 - 5*q^34 - 6*q^32 - 7*q^30 - 8*q^28 - 9*q^26 - 10*q^24 - 10*q^22 - 10*q^20 - 9*q^18 - 8*q^16 - 7*q^14 - 6*q^12 - 5*q^10 - 4*q^8 - 3*q^6 - 2*q^4 - q^2,
 -q^50 - 2*q^48 - 3*q^46 - 4*q^44 - 5*q^42 - 6*q^40 - 7*q^38 - 8*q^36 - 9*q^34 - 10*q^32 - 11*q^30 - 12*q^28 - 12*q^26 - 12*q^24 - 11*q^22 - 10*q^20 - 9*q^18 - 8*q^16 - 7*q^14 - 6*q^12 - 5*q^10 - 4*q^8 - 3*q^6 - 2*q^4 - q^2,
 -q^58 - 2*q^56 - 3*q^54 - 4*q^52 - 5*q^50 - 6*q^48 - 7*q^46 - 8*q^44 - 9*q^42 - 10*q^40 - 11*q^38 - 12*q^36 - 13*q^34 - 14*q^32 - 14*q^30 - 14*q^28 - 13*q^26 - 12*q^24 - 11*q^22 - 10*q^20 - 9*q^18 - 8*q^16 - 7*q^14 - 6*q^12 - 5*q^10 - 4*q^8 - 3*q^6 - 2*q^4 - q^2,
 -q^66 - 2*q^64 - 3*q^62 - 4*q^60 - 5*q^58 - 6*q^56 - 7*q^54 - 8*q^52 - 9*q^50 - 10*q^48 - 11*q^46 - 12*q^44 - 13*q^42 - 14*q^40 - 15*q^38 - 16*q^36 - 16*q^34 - 16*q^32 - 15*q^30 - 14*q^28 - 13*q^26 - 12*q^24 - 11*q^22 - 10*q^20 - 9*q^18 - 8*q^16 - 7*q^14 - 6*q^12 - 5*q^10 - 4*q^8 - 3*q^6 - 2*q^4 - q^2]

In [3]:
get_matrix(B.as_2dim())

[                                                                                                                                      1                                                                                                                                       1                                                                                                                                       1                                                                                                                                       1                                                                                                                                       1]
[                                                                                                                                      0                                                                                                                                 q^2 + 1                                                                                                                     q^6 + q^4 + q^2 + 1                                                                                                        q^10 + q^8 + q^6 + q^4 + q^2 + 1                                                                                          q^14 + q^12 + q^10 + q^8 + q^6 + q^4 + q^2 + 1]
[                                                                                                                                      0                                                                                                                                       1                                                                                                             q^8 + q^6 + 2*q^4 + q^2 + 1                                                                         q^16 + q^14 + 2*q^12 + 2*q^10 + 3*q^8 + 2*q^6 + 2*q^4 + q^2 + 1                                     q^24 + q^22 + 2*q^20 + 2*q^18 + 3*q^16 + 3*q^14 + 4*q^12 + 3*q^10 + 3*q^8 + 2*q^6 + 2*q^4 + q^2 + 1]
[                                                                                                                                      0                                                                                                                                       0                                                                                                                     q^6 + q^4 + q^2 + 1                                                                q^18 + q^16 + 2*q^14 + 3*q^12 + 3*q^10 + 3*q^8 + 3*q^6 + 2*q^4 + q^2 + 1          q^30 + q^28 + 2*q^26 + 3*q^24 + 4*q^22 + 5*q^20 + 6*q^18 + 6*q^16 + 6*q^14 + 6*q^12 + 5*q^10 + 4*q^8 + 3*q^6 + 2*q^4 + q^2 + 1]
[                                                                                                                                      0                                                                                                                                       0                                                                                                                                       1                                                                         q^16 + q^14 + 2*q^12 + 2*q^10 + 3*q^8 + 2*q^6 + 2*q^4 + q^2 + 1 q^32 + q^30 + 2*q^28 + 3*q^26 + 5*q^24 + 5*q^22 + 7*q^20 + 7*q^18 + 8*q^16 + 7*q^14 + 7*q^12 + 5*q^10 + 5*q^8 + 3*q^6 + 2*q^4 + q^2 + 1]

In [4]:
all(
    q_binomial(2*n,k,q^2)
    ==
    B[k](n)
    for n in range(10)
    for k in range(10)
)

True

In [5]:
B.compatibility("q_4n")

Compatibility condition (0, 1, 1) with following coefficient matrix:
[            q_k^2 q^2*q_k^4 - q_k^2]

In [6]:
all(
    q**(4*n) * B[k](n)
    ==
    q**(2*k) * B[k](n) + (q**(4*k+2) - q**(2*k))*B[k+1](n)
    for n in range(10)
    for k in range(10)
)

True

In [7]:
B.compatibility("q_4n")[0,1].universe

Fraction Field of Multivariate Polynomial Ring in q, x over Rational Field

In [8]:
B.compatibility("E")

Compatibility condition (2, 0, 1) with following coefficient matrix:
[                    1 ((q^2 + 1)/q^2)*q_k^2                 q_k^4]

In [9]:
all(
    B[k](n+1)
    ==
    B[k-2](n) + ((q^2+1)*q**(2*k))/q**2 * B[k-1](n) + q**(4*k)*B[k](n)
    for n in range(10)
    for k in range(2,10)
)

True

In [10]:
B.recurrence("E")

Sk^2 + ((q^2 + 1)*q_k^2)*Sk + q_k^4

Now we use the operator with $q^{4n}$ that annihilates the sequence $a(n)$ described above:

In [11]:
S = B.recurrence("\
    -q^2*x^2*E^2 + \
    (x^2 + q^2*x^2 + q^4*q_4n*x^2 + q^6*q_4n*x^2)*E + \
    q_4n*q^2 - x^2 - q^4*q_4n^2*x^2 + q^2*q_4n*x^4\
", output="ore")
S

-q^2*x^2*Sk^6 + (((-q^12 - q^10 - q^8 - q^6)*x^2)*q_k^2)*Sk^5 + (((-q^18 - q^16 - 2*q^14 - q^12 - q^10)*x^2)*q_k^4 + ((q^10 + q^8)*x^2)*q_k^2 + (q^2 + 1)*x^2)*Sk^4 + (((-q^20 - q^18 - q^16 - q^14)*x^2)*q_k^6 + ((q^16 + 2*q^14 + 2*q^12 + q^10)*x^2)*q_k^4 + ((q^6 + q^4)*x^2)*q_k^2)*Sk^3 + (-q^18*x^2*q_k^8 + ((q^18 + 2*q^16 + 2*q^14 + q^12)*x^2)*q_k^6 + ((-2*q^12 - q^10)*x^2)*q_k^4 + (q^6*x^4 + q^6)*q_k^2 - x^2)*Sk^2 + (((q^16 + q^14)*x^2)*q_k^8 + ((-q^14 - 2*q^12 - q^10)*x^2)*q_k^6 + (q^8*x^4 + (q^10 + q^8)*x^2 + q^8)*q_k^4 + (-q^4*x^4 - q^4)*q_k^2)*Sk - q^12*x^2*q_k^8 + ((q^10 + q^8)*x^2)*q_k^6 - q^6*x^2*q_k^4

In [12]:
from ore_algebra import OreAlgebra
new_base = S.parent().base().base().extend_variables(["q_kk", "x_k","q_n", "q_nk"]).fraction_field()
q_kk = new_base("q_kk"); q_k = new_base("q_k")
q_n = new_base("q_n"); q_nk = new_base("q_nk"); 
x_k = new_base("x_k"); x = new_base("x"); q = new_base("q")

In [13]:
new_OS = OreAlgebra(new_base, ("Sk", lambda f : f(q_kk=q_kk*q_k**2*q, q_nk=q_nk*q_n, q_k=q*q_k, x_k=x*x_k), lambda _ : 0))
Sk = new_OS.gens()[0]

In [14]:
new_S = new_OS(S)
el = q_kk/q_nk^2*x_k; el

q_kk*x_k/q_nk^2

In [15]:
out = (new_S * el).polynomial()(Sk=1)
out

((-q^22*x^4)*q_k^12*q_kk*x_k*q_n^38*q_nk^12 + ((q^17 + q^15)*x^3)*q_k^10*q_kk*x_k*q_n^40*q_nk^12 + (-q^12*x^2)*q_k^8*q_kk*x_k*q_n^42*q_nk^12 + ((-q^29 - q^27 - q^25 - q^23)*x^5)*q_k^12*q_kk*x_k*q_n^36*q_nk^12 + ((q^22 + 2*q^20 + 2*q^18 + q^16)*x^4)*q_k^10*q_kk*x_k*q_n^38*q_nk^12 + ((-q^15 - 2*q^13 - q^11)*x^3)*q_k^8*q_kk*x_k*q_n^40*q_nk^12 + ((q^10 + q^8)*x^2)*q_k^6*q_kk*x_k*q_n^42*q_nk^12 + ((-q^34 - q^32 - 2*q^30 - q^28 - q^26)*x^6)*q_k^12*q_kk*x_k*q_n^34*q_nk^12 + ((q^25 + 2*q^23 + 2*q^21 + q^19)*x^5)*q_k^10*q_kk*x_k*q_n^36*q_nk^12 + ((-2*q^16 - q^14)*x^4)*q_k^8*q_kk*x_k*q_n^38*q_nk^12 + (q^9*x^5 + (q^11 + q^9)*x^3 + q^9*x)*q_k^6*q_kk*x_k*q_n^40*q_nk^12 + (-q^6*x^2)*q_k^4*q_kk*x_k*q_n^42*q_nk^12 + ((-q^37 - q^35 - q^33 - q^31)*x^7)*q_k^12*q_kk*x_k*q_n^32*q_nk^12 + ((q^26 + q^24)*x^6)*q_k^10*q_kk*x_k*q_n^34*q_nk^12 + ((q^15 + q^13)*x^5)*q_k^8*q_kk*x_k*q_n^36*q_nk^12 + (q^10*x^6 + q^10*x^2)*q_k^6*q_kk*x_k*q_n^38*q_nk^12 + (-q^5*x^5 - q^5*x)*q_k^4*q_kk*x_k*q_n^40*q_nk^12 + (-q^38*x^8)*q_k^12*q_kk*x_k*q_n^30*q_nk^12 + ((q^18 + q^16)*x^6)*q_k^8*q_kk*x_k*q_n^34*q_nk^12 + (-q^4*x^4)*q_k^4*q_kk*x_k*q_n^38*q_nk^12)/(q_n^42*q_nk^14)

In [16]:
sq = SR("q"); sn = SR("n"); sk = SR("k"); sx = SR("x")
show(out.parent())
sout = SR(out)

Fraction Field of Multivariate Polynomial Ring in q_k, q_kk, x_k, q_n, q_nk over Fraction Field of Univariate Polynomial Ring in x over Fraction Field of Univariate Polynomial Ring in q over Rational Field

In [17]:
sout = SR(out)(q_k=sq**sk, q_kk=sq**(sk**2), x_k=sx**sk, q_n=sq**sn, q_nk=sq**(sn*sk))

In [18]:
sout.simplify_full()

-((q^37 + q^35 + q^33 + q^31)*q^(k^2 + 12*k + 2*n)*x^7 + q^(k^2 + 12*k + 38)*x^8 + (((q^34 + q^32 + 2*q^30 + q^28 + q^26)*q^(12*k) - (q^26 + q^24)*q^(10*k) - (q^18 + q^16)*q^(8*k))*q^(k^2 + 4*n) - q^(k^2 + 6*k + 8*n + 10))*x^6 - ((q^22 + 2*q^20 + 2*q^18 + q^16)*q^(10*k) - (2*q^16 + q^14)*q^(8*k) - q^(12*k + 22) - q^(4*k + 4))*q^(k^2 + 8*n)*x^4 + (((q^29 + q^27 + q^25 + q^23)*q^(12*k) - (q^25 + 2*q^23 + 2*q^21 + q^19)*q^(10*k) - (q^15 + q^13)*q^(8*k))*q^(k^2 + 6*n) - q^(k^2 + 10*n)*(q^(6*k + 9) - q^(4*k + 5)))*x^5 - ((q^17 + q^15)*q^(10*k) - (q^15 + 2*q^13 + q^11)*q^(8*k) + (q^11 + q^9)*q^(6*k))*q^(k^2 + 10*n)*x^3 - q^(k^2 + 10*n)*(q^(6*k + 9) - q^(4*k + 5))*x - (((q^10 + q^8)*q^(6*k) - q^(8*k + 12) - q^(4*k + 6))*q^(k^2 + 12*n) + q^(k^2 + 6*k + 8*n + 10))*x^2)*q^(-2*k*n - 12*n)*x^k

In [19]:
required_init(S)

6

In [20]:
SD = B.recurrence("\
    -q^2*x^2*E^2 + \
    (x^2 + q^2*x^2 + q^4*q_4n*x^2 + q^6*q_4n*x^2)*E + \
    q_4n*q^2 - x^2 - q^4*q_4n^2*x^2 + q^2*q_4n*x^4\
", output="ore_double")
SD

(-q^2*x^2)*Sk^4 + (((-q^8 - q^6 - q^4 - q^2)*x^2)*q_k^2)*Sk^3 + (((-q^10 - q^8 - 2*q^6 - q^4 - q^2)*x^2)*q_k^4 + ((q^6 + q^4)*x^2)*q_k^2 + (q^2 + 1)*x^2)*Sk^2 + (-1/q^4*x^2*q_k^8 + (((q^2 + 1)/q^4)*x^2)*q_k^6 - 1/q^2*x^2*q_k^4)*Ski^2 + (((-q^8 - q^6 - q^4 - q^2)*x^2)*q_k^6 + ((q^8 + 2*q^6 + 2*q^4 + q^2)*x^2)*q_k^4 + ((q^2 + 1)*x^2)*q_k^2)*Sk + ((((q^2 + 1)/q^2)*x^2)*q_k^8 + (((-q^4 - 2*q^2 - 1)/q^2)*x^2)*q_k^6 + (x^4 + (q^2 + 1)*x^2 + 1)*q_k^4 + (-x^4 - 1)*q_k^2)*Ski - q^2*x^2*q_k^8 + ((q^6 + 2*q^4 + 2*q^2 + 1)*x^2)*q_k^6 + ((-2*q^4 - q^2)*x^2)*q_k^4 + (q^2*x^4 + q^2)*q_k^2 - x^2

In [21]:
## This was the lhs of our identity
x = B.base("x"); q=B.base("q")
a = Q_pochhammer(-q*x, q^2) * Q_pochhammer(-q/x, q^2)
## We build the matrix that gives the initial condition problem
M = Matrix([[B[k](n) for n in range(4)] for k in range(7)])
K = M.left_kernel()
init = M.solve_left(vector([a(i) for i in range(M.ncols())]))

In [22]:
K

Vector space of degree 7 and dimension 3 over Fraction Field of Multivariate Polynomial Ring in q, x over Rational Field
Basis matrix:
[                                                                                                        0                                                                                                         1                                                                                                  -q^2 - 1                                                                                                         0                                                                        q^10 + 2*q^8 + 2*q^6 + 2*q^4 + q^2                                                                                                         0 -q^26 - 3*q^24 - 6*q^22 - 10*q^20 - 13*q^18 - 15*q^16 - 15*q^14 - 13*q^12 - 10*q^10 - 6*q^8 - 3*q^6 - q^4]
[                                                                                                        0                                                                                                         0                                                                                                         0                                                                                                         1                                                                                      -q^6 - q^4 - q^2 - 1                                                                                                         0                  q^22 + 2*q^20 + 3*q^18 + 5*q^16 + 6*q^14 + 6*q^12 + 6*q^10 + 5*q^8 + 3*q^6 + 2*q^4 + q^2]
[                                                                                                        0                                                                                                         0                                                                                                         0                                                                                                         0                                                                                                         0                                                                                                         1                                                                         -q^10 - q^8 - q^6 - q^4 - q^2 - 1]

In [23]:
init

(1, ((q/(q^14 + 2*q^12 + 2*q^10 + 3*q^8 + 3*q^6 + 2*q^4 + 2*q^2 + 1))*x^6 + ((-q^6 + q^2 + 1)/(q^12 + 3*q^10 + 4*q^8 + 4*q^6 + 3*q^4 + q^2))*x^5 + ((q^8 + q^6 + q^4 + q^2 + 1)/(q^9 + 3*q^7 + 3*q^5 + q^3))*x^4 + ((q^16 + 2*q^12 + 2*q^8 + q^6 + q^4 + 2)/(q^16 + 2*q^14 + 2*q^12 + 3*q^10 + 3*q^8 + 2*q^6 + 2*q^4 + q^2))*x^3 + ((q^8 + q^6 + q^4 + q^2 + 1)/(q^9 + 3*q^7 + 3*q^5 + q^3))*x^2 + ((-q^6 + q^2 + 1)/(q^12 + 3*q^10 + 4*q^8 + 4*q^6 + 3*q^4 + q^2))*x + q/(q^14 + 2*q^12 + 2*q^10 + 3*q^8 + 3*q^6 + 2*q^4 + 2*q^2 + 1))/x^3, ((-q/(q^12 + q^10 + q^8 + 2*q^6 + q^4 + q^2 + 1))*x^6 + ((q^6 - q^2 - 1)/(q^10 + 2*q^8 + 2*q^6 + 2*q^4 + q^2))*x^5 + ((q^6 - q^2 - 1)/(q^7 + 2*q^5 + q^3))*x^4 + ((q^14 - q^12 + 2*q^10 - q^8 - 2)/(q^14 + q^12 + q^10 + 2*q^8 + q^6 + q^4 + q^2))*x^3 + ((q^6 - q^2 - 1)/(q^7 + 2*q^5 + q^3))*x^2 + ((q^6 - q^2 - 1)/(q^10 + 2*q^8 + 2*q^6 + 2*q^4 + q^2))*x - q/(q^12 + q^10 + q^8 + 2*q^6 + q^4 + q^2 + 1))/x^3, ((q^3/(q^12 + q^10 + q^8 + 2*q^6 + q^4 + q^2 + 1))*x^6 + ((q^4 + q^2 + 1)/(q^8 + 2*q^6 + 2*q^4 + 2*q^2 + 1))*x^5 + ((q^4 + q^2 + 1)/(q^5 + 2*q^3 + q))*x^4 + ((q^12 + 2*q^8 + 2*q^4 + 2)/(q^12 + q^10 + q^8 + 2*q^6 + q^4 + q^2 + 1))*x^3 + ((q^4 + q^2 + 1)/(q^5 + 2*q^3 + q))*x^2 + ((q^4 + q^2 + 1)/(q^8 + 2*q^6 + 2*q^4 + 2*q^2 + 1))*x + q^3/(q^12 + q^10 + q^8 + 2*q^6 + q^4 + q^2 + 1))/x^3, 0, 0, 0)

In [24]:
c = solution(SD, init)

In [25]:
a2 = Sequence(lambda n : sum(c(k)*B[k](n) for k in range(2*n+1)), universe=B.base, dim=1)

In [26]:
[a2(n) == a(n) for n in range(10)]

[True, True, True, False, False, False, False, False, False, False]

$$a_n(x) = \left(\frac{-q}{x};q^2\right)_n \left(-xq;q^2\right)_n$$

In [27]:
S

-q^2*x^2*Sk^6 + (((-q^12 - q^10 - q^8 - q^6)*x^2)*q_k^2)*Sk^5 + (((-q^18 - q^16 - 2*q^14 - q^12 - q^10)*x^2)*q_k^4 + ((q^10 + q^8)*x^2)*q_k^2 + (q^2 + 1)*x^2)*Sk^4 + (((-q^20 - q^18 - q^16 - q^14)*x^2)*q_k^6 + ((q^16 + 2*q^14 + 2*q^12 + q^10)*x^2)*q_k^4 + ((q^6 + q^4)*x^2)*q_k^2)*Sk^3 + (-q^18*x^2*q_k^8 + ((q^18 + 2*q^16 + 2*q^14 + q^12)*x^2)*q_k^6 + ((-2*q^12 - q^10)*x^2)*q_k^4 + (q^6*x^4 + q^6)*q_k^2 - x^2)*Sk^2 + (((q^16 + q^14)*x^2)*q_k^8 + ((-q^14 - 2*q^12 - q^10)*x^2)*q_k^6 + (q^8*x^4 + (q^10 + q^8)*x^2 + q^8)*q_k^4 + (-q^4*x^4 - q^4)*q_k^2)*Sk - q^12*x^2*q_k^8 + ((q^10 + q^8)*x^2)*q_k^6 - q^6*x^2*q_k^4

In [28]:
B_wc = S.parent().base().base().change_ring(
    S.parent().base().base().base().base().extend_variables(
        ["c_0","c_1","c_2","c_3","c_4","c_5"]
    ).fraction_field()
).fraction_field()
q_k = B_wc("q_k"); 
x = B_wc.base().base().base().gens()[0]; c = B_wc.base().base().base().gens()[1:]
q = B_wc.base().base().base().base().base().gens()[0]
OA_wc = OreAlgebra(B_wc, ("Sk", lambda p: p(q_k=q*q_k), lambda _ : 0))
Sk = OA_wc("Sk")
S_wc = OA_wc(S)

In [29]:
generic_c = solution(S_wc, c)

In [30]:
generic_c(7)

((q^20 + q^18 - q^12 - q^10)*x^4*c_1 + (q^18 + q^16 + q^14 + q^6)*x^4*c_2 + (-q^8)*x^4*c_3 + (-q^24 + q^20 + q^16 - q^12)*x^2*c_0 + (q^28 + q^26 - q^24 - q^22 - q^18 + q^10)*x^2*c_1 + (2*q^28 + 4*q^26 + 2*q^24 + q^22 + q^20 - q^14 - 2*q^12 - q^10 - q^8 - q^6)*x^2*c_2 + (-q^32 - 2*q^30 - 2*q^28 + q^24 + 2*q^22 + 2*q^20 + 3*q^18 + 5*q^16 + 3*q^14 + 2*q^12 + q^10 + 1)*x^2*c_3 + (-q^30 - 2*q^28 - 3*q^26 - 4*q^24 - 3*q^22 - 2*q^20 - 2*q^18 - q^16 + q^14 + 2*q^12 + 2*q^10 + q^8)*x^2*c_4 + (-q^24 - q^22 - 2*q^20 - 2*q^18 - 2*q^16 - q^14 - 2*q^12 - q^10 - q^2 - 1)*x^2*c_5 + (q^20 + q^18 - q^12 - q^10)*c_1 + (q^18 + q^16 + q^14 + q^6)*c_2 + (-q^8)*c_3)/((-q^2)*x^2)

In [31]:
k = 3
num = generic_c(k).numerator()
denom = generic_c(k).denominator()
all(num.degree(c[i]) in (0,1) for i in range(len(c))) and all(denom.degree(c[i]) == 0 for i in range(len(c)))

True

In [32]:
num == sum(c[i]*num.coefficient(c[i]) for i in range(len(c)))

True

In [58]:
n = len(c)
columns = []
for k in range(2*n-1):
    num = generic_c(k).numerator()
    denom = generic_c(k).denominator()
    columns.append([num.coefficient(c[i])/denom for i in range(len(c))])
N = Matrix(columns).transpose()

In [59]:
M = Matrix([[B[k](n) for n in range(n)] for k in range(2*n-1)])

In [60]:
(N*M)

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          (-q^10 + q^8 + q^6 - q^4 + 1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   (-q^22 + q^18 - q^14 + q^12 + 2*q^6 - q^4 - q^2 + 1)                                                                                                                                                                                                                                                                                                                                                                                                                                      ((-q^26 + q^24 + q^22 - q^20)*x^4 + (-q^42 + q^26 - q^22 + q^20 + q^14 + q^12 + q^10 - q^8 - q^6)*x^2 + (-q^26 + q^24 + q^22 - q^20))/((q^4)*x^2)]
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     0                             

In [61]:
init = (N*M).solve_left(vector(a(k) for k in range(n)))

ValueError: matrix equation has no solutions

## **Product basis of q-factorial bases** (<span style="color:green">TODO: review this section</span>)

Let us consider the following 2 $q$-basis:
$$\left\{P_k(n) = q^{4nk}\right\}_k,\qquad\left\{B_k(n) = \qbinomb{2n}{k}{q^2}\right\}_k.$$

These two bases are $(q^4)$-factorial and compatible with $E: n\mapsto (n+1)$. So we would like to do a constructions similar to the **product basis** as was done for *factorial bases* in the paper [doi:10.1016/j.jsc.2022.11.002](https://doi.org/10.1016/j.jsc.2022.11.002):

$$Q_{k}(n) = \left\{\begin{array}{lcl}
    P_{l}(n)B_{l}(n)& \text{if} & k = 2l\\
    P_{l+1}(n) B_{l}(n) & \text{if} & k = 2l + 1
\end{array}\right.$$

We can then start analyzing the compatibilities with $q^{4n}$ and $E$.

In [2]:
B = QBinomialBasis(2,e=2)
P = QPowerBasis(4)

In [3]:
Q = QBasis(
    lambda k : P[k//2 + k%2]*B[k//2], 
    P.base)
Q = Q.change_base(Q.base.base().extend_variables('x').fraction_field())
x = Q.base("x"); q=Q.q
q_k = B.compatibility("q_4n").change_base(Q.base)[0,0].extra_info()["variables"][0]

### Compatibility with $q^{4n}$

In [4]:
P.compatibility("q_4n")

Compatibility condition (0, 1, 1) with following coefficient matrix:
[0 1]

In [5]:
B.compatibility("q_4n")

Compatibility condition (0, 1, 1) with following coefficient matrix:
[            q_k^2 q^2*q_k^4 - q_k^2]

##### * **$k = 2l$**

$$q^{4n}Q_{2l}(n) = q^{4n}P_l(n)B_l(n) = P_{l+1}(n)B_l(n) = Q_{2l+1}(n).$$

##### * $k = 2l+1$


$$q^{4n}Q_{2l+1}(n) = q^{4n}P_{l+1}(n)B_l(n) = P_{l+1}\left(q^{2l}B_l(n) + (q^{4l+2}-q^{2l})B_{l+1}(n)\right) = q^{2l}Q_{2l+1}(n) + (q^{4l+2}-q^{2l})Q_{2l+2}(n)$$

So, in particular, $Q_k(n)$ is $(q^4)$-factorial and it has a compatibility in two sections.

In [6]:
Q.set_compatibility("q_4n", Compatibility([
    [P.compatibility("q_4n").change_base(Q.base)[0,0], P.compatibility("q_4n").change_base(Q.base)[0,1]],
    [B.compatibility("q_4n").change_base(Q.base)[0,0], B.compatibility("q_4n").change_base(Q.base)[0,1]]], 
    0, 1, 2), True, "any")

Compatibility condition (0, 1, 2) with following coefficient matrix:
[                0                 1]
[            q_k^2 q^2*q_k^4 - q_k^2]

In [7]:
Q.recurrence("q_4n")

[                                   0 (1/q^2*q_k^4 + ((-1)/q^2)*q_k^2)*Ski]
[                                   1                                q_k^2]

In [8]:
check_compatibility(Q, "q_4n", lambda seq : Qn**4*seq, 30)

True

### Compatibility with $E$

In [9]:
B.compatibility("E")

Compatibility condition (2, 0, 1) with following coefficient matrix:
[                    1 ((q^2 + 1)/q^2)*q_k^2                 q_k^4]

In [10]:
P.compatibility("E")

Compatibility condition (0, 0, 1) with following coefficient matrix:
[q_k^4]

##### * **$k = 2l$**

$$\begin{array}{rl}
Q_{2l}(n+1) & = 
    P_l(n+1)B_l(n+1) = q^{4l} P_l(n) \left(B_{l-2}(n) + \frac{q^{2l}(q^2 + 1)}{q^2} B_{l-1}(n) + q^{4l}B_l(n)\right)\\
    & = q^{4l}P_l(n)B_{l-2}(n) + \frac{q^{6l}(q^2 + 1)}{q^2} P_l(n) B_{l-1}(n) + q^{8l} P_l(n)B_l(n) \\
    & = q^{4l}\frac{P_l(n)}{P_{l-1}(n)}P_{l-1}(n)B_{l-2}(n) + \frac{q^{6l}(q^2 + 1)}{q^2} P_l(n) B_{l-1}(n) + q^{8l} P_l(n)B_l(n)\\
    & = q^{4l}q^{4n}Q_{2l-3}(n) + \frac{q^{6l}(q^2 + 1)}{q^2} Q_{2l-1}(n) + q^{8l} Q_l(n)\\
    & = q^{6l-4}Q_{2l-3}(n) + (q^{8l-6} - q^{6l-4})Q_{2l-2}(n) + \frac{q^{6l}(q^2 + 1)}{q^2} Q_{2l-1}(n) + q^{8l} Q_l(n)
\end{array}$$

##### * **$k = 2l + 1$**

$$\begin{array}{rl}
Q_{2l+1}(n+1) & = 
    P_{l+1}(n+1)B_l(n+1) = q^{4l+4} P_{l+1}(n) \left(B_{l-2}(n) + \frac{q^{2l}(q^2 + 1)}{q^2} B_{l-1}(n) + q^{4l}B_l(n)\right)\\
    & = ...\\
    & = q^{8l-4}Q_{2l-3}(n) + \left(q^{10l - 6} - q^{8l - 4}\right)Q_{2l-2}(n) \\
    & + \left(2q^{8l-2} - q^{6l} + q^{8l} \right)Q_{2l-1}(n) + q^{8l}\left(q^{2l+2} + q^{2l} - q^2 - 1\right)Q_{2l}(n) + q^{8l+4}Q_{2l+1}(n)
\end{array}$$

In [11]:
coeffs = [
    [
        0, 
        q_k^6/q**4, 
        q_k^6/q**6*(q_k^2 - q^2), 
        q_k^6/q**2 * (q** 2+1), 
        q_k^8
    ],
    [
        q_k**8/q**4,
        q_k**10/q**6 - q_k**8/q**4,
        q_k**8/q**2 - q_k**6 + q_k**8*(q**2+1),
        q_k**6*q**2*(q_k**4/q**2 - q_k**2/q**2)*(q**2+1),
        q_k**8*q**4
    ]
]
coeffs = [[QRationalSequence(el, variables=[q_k], universe=Qn.universe, q=Qn.q) for el in row] for row in coeffs]

In [12]:
Q.set_homomorphism("E", Compatibility(coeffs, 4, 0, 2), True)

Compatibility condition (4, 0, 2) with following coefficient matrix:
[                                  0                         1/q^4*q_k^6           1/q^6*q_k^8 - 1/q^4*q_k^6               ((q^2 + 1)/q^2)*q_k^6                               q_k^8]
[                        1/q^4*q_k^8          1/q^6*q_k^10 - 1/q^4*q_k^8 ((q^4 + q^2 + 1)/q^2)*q_k^8 - q_k^6 (q^2 + 1)*q_k^10 + (-q^2 - 1)*q_k^8                           q^4*q_k^8]

In [13]:
Q.recurrence("E")

[                                     (q^2*q_k^8 - q^2*q_k^6)*Sk + q_k^8       (q^4*q_k^10 - q^4*q_k^8)*Sk + (q^2 + 1)*q_k^10 + (-q^2 - 1)*q_k^8]
[                                q^8*q_k^6*Sk^2 + ((q^6 + q^4)*q_k^6)*Sk q^12*q_k^8*Sk^2 + ((q^10 + q^8 + q^6)*q_k^8 - q^6*q_k^6)*Sk + q^4*q_k^8]

In [14]:
check_compatibility(Q, "E", lambda seq: seq.shift(), 30)

True

### Applying the basis

In [15]:
M = Q.recurrence("E + (-1 - q_4n*q*x - q_4n*q^3*x - q_4n^2*q^4*x^2)", output="ore")

In [16]:
M

[                                                                             (q^10*q_k^8 - q^8*q_k^6)*Sk^2 + (q^8*q_k^8 - 1)*Sk - q^6*x^2*q_k^4 + q^4*x^2*q_k^2      (q^14*q_k^10 - q^12*q_k^8)*Sk^2 + ((q^12 + q^10)*q_k^10 + (-q^10 - q^8)*q_k^8)*Sk - q^6*x^2*q_k^6 + (-q^5*x + q^4*x^2 - q^3*x)*q_k^4 + (q^3*x + q*x)*q_k^2]
[                                                                               q^14*q_k^6*Sk^3 + ((q^12 + q^10)*q_k^6)*Sk^2 + (-q^6*x^2*q_k^2 - q^3*x - q*x)*Sk q^20*q_k^8*Sk^3 + ((q^18 + q^16 + q^14)*q_k^8 - q^12*q_k^6)*Sk^2 + (q^12*q_k^8 - q^8*x^2*q_k^4 + (-q^5*x - q^3*x)*q_k^2 - 1)*Sk - q^6*x^2*q_k^4 + q^4*x^2*q_k^2]

In [17]:
M[0,0].gcrd(M[1,0])

1

In [18]:
M[0,1].gcrd(M[1,1])

1

### One sec: we can do it without product basis

In [2]:
B = QBinomialBasis(2, e=2)
B = B.change_base(B.base.base().extend_variables('x').fraction_field())
x = B.base("x"); q=B.q

In [4]:
basis_matrix(B)

[                                                                                                                                      1                                                                                                                                       1                                                                                                                                       1                                                                                                                                       1                                                                                                                                       1]
[                                                                                                                                      0                                                                                                                                 q^2 + 1                                                                                                                     q^6 + q^4 + q^2 + 1                                                                                                        q^10 + q^8 + q^6 + q^4 + q^2 + 1                                                                                          q^14 + q^12 + q^10 + q^8 + q^6 + q^4 + q^2 + 1]
[                                                                                                                                      0                                                                                                                                       1                                                                                                             q^8 + q^6 + 2*q^4 + q^2 + 1                                                                         q^16 + q^14 + 2*q^12 + 2*q^10 + 3*q^8 + 2*q^6 + 2*q^4 + q^2 + 1                                     q^24 + q^22 + 2*q^20 + 2*q^18 + 3*q^16 + 3*q^14 + 4*q^12 + 3*q^10 + 3*q^8 + 2*q^6 + 2*q^4 + q^2 + 1]
[                                                                                                                                      0                                                                                                                                       0                                                                                                                     q^6 + q^4 + q^2 + 1                                                                q^18 + q^16 + 2*q^14 + 3*q^12 + 3*q^10 + 3*q^8 + 3*q^6 + 2*q^4 + q^2 + 1          q^30 + q^28 + 2*q^26 + 3*q^24 + 4*q^22 + 5*q^20 + 6*q^18 + 6*q^16 + 6*q^14 + 6*q^12 + 5*q^10 + 4*q^8 + 3*q^6 + 2*q^4 + q^2 + 1]
[                                                                                                                                      0                                                                                                                                       0                                                                                                                                       1                                                                         q^16 + q^14 + 2*q^12 + 2*q^10 + 3*q^8 + 2*q^6 + 2*q^4 + q^2 + 1 q^32 + q^30 + 2*q^28 + 3*q^26 + 5*q^24 + 5*q^22 + 7*q^20 + 7*q^18 + 8*q^16 + 7*q^14 + 7*q^12 + 5*q^10 + 5*q^8 + 3*q^6 + 2*q^4 + q^2 + 1]

In [5]:
B.compatibility("q_4n")

Compatibility condition (0, 1, 1) with following coefficient matrix:
[            q_k^2 q^2*q_k^4 - q_k^2]

In [6]:
B.recurrence("q_4n")

2023-08-29 11:54:18 DEBUG    [recurrence] --- STARTING COMPUTATION OF A RECURRENCE
2023-08-29 11:54:18 DEBUG    [recurrence] --- FINISHED COMPUTATION OF A DICTIONARY REPRESENTATION
2023-08-29 11:54:18 DEBUG    [recurrence] Processing a single recurrence
2023-08-29 11:54:18 DEBUG    [recurrence] Output is ore-related (output='ore_double')
2023-08-29 11:54:18 DEBUG    [recurrence] Processing a single recurrence
2023-08-29 11:54:18 DEBUG    [recurrence] output='rational'
2023-08-29 11:54:18 DEBUG    [recurrence] Processing a double-recurrence operator
2023-08-29 11:54:18 DEBUG    [recurrence] --- FINISHED COMPUTATION OF THE FINAL RECURRENCE
recurrence=(1/q^2*q_k^4 + ((-1)/q^2)*q_k^2)*Ski + q_k^2


(1/q^2*q_k^4 + ((-1)/q^2)*q_k^2)*Ski + q_k^2

In [7]:
op = B.recurrence("E + (-1 - q_4n*q*x - q_4n*q^3*x - q_4n^2*q^4*x^2)", output="ore")

2023-08-29 11:54:19 DEBUG    [recurrence] --- STARTING COMPUTATION OF A RECURRENCE
2023-08-29 11:54:19 DEBUG    [recurrence] --- FINISHED COMPUTATION OF A DICTIONARY REPRESENTATION
2023-08-29 11:54:19 DEBUG    [recurrence] Processing a single recurrence
2023-08-29 11:54:19 DEBUG    [recurrence] Output is ore-related (output='ore')
2023-08-29 11:54:19 DEBUG    [recurrence] Processing a single recurrence
2023-08-29 11:54:19 DEBUG    [recurrence] output='rational'
2023-08-29 11:54:19 DEBUG    [recurrence] Processing a recurrence operator
2023-08-29 11:54:19 DEBUG    [recurrence] --- FINISHED COMPUTATION OF THE FINAL RECURRENCE
recurrence=Sk^4 + ((q^6 + q^4)*q_k^2)*Sk^3 + ((-q^12*x^2 + q^8)*q_k^4 + (-q^7*x - q^5*x)*q_k^2 - 1)*Sk^2 + ((-q^14*x^2 - q^12*x^2)*q_k^6 + (q^10*x^2 - q^9*x + q^8*x^2 - q^7*x)*q_k^4 + (q^5*x + q^3*x)*q_k^2)*Sk - q^12*x^2*q_k^8 + (q^10*x^2 + q^8*x^2)*q_k^6 - q^6*x^2*q_k^4


In [8]:
op

Sk^4 + ((q^6 + q^4)*q_k^2)*Sk^3 + ((-q^12*x^2 + q^8)*q_k^4 + (-q^7*x - q^5*x)*q_k^2 - 1)*Sk^2 + ((-q^14*x^2 - q^12*x^2)*q_k^6 + (q^10*x^2 - q^9*x + q^8*x^2 - q^7*x)*q_k^4 + (q^5*x + q^3*x)*q_k^2)*Sk - q^12*x^2*q_k^8 + (q^10*x^2 + q^8*x^2)*q_k^6 - q^6*x^2*q_k^4

In [11]:
required_init(op)

4

In [12]:
q = B.q; x = B.base("x")
what_c = solution(op, [1, q*x, q^4*x^2, q^9*x^3])

In [13]:
what_c[:10]

[1,
 q*x,
 q^4*x^2,
 q^9*x^3,
 q^16*x^4,
 q^25*x^5,
 q^36*x^6,
 q^49*x^7,
 q^64*x^8,
 q^81*x^9]

In [14]:
Sk, q_k, q, x = [op.parent()(el) for el in ("Sk", "q_k", "q", "x")]

In [15]:
op2 = (Sk - q_k^2*q*x)

In [16]:
op.gcrd(op2)

Sk - q*x*q_k^2

## **A new example for products**

Now we would like to build the product of these two bases:
$$\left\{P_k(n) = q^{nk}\right\},\qquad \left\{B_k(n) = \qbinom{n}{k}\right\}.$$

In this case, both are $q$-factorial bases, meaning we can build the corresponding product basis.

In [2]:
P = QPowerBasis()
B = QBinomialBasis()

In [3]:
Q = QBasis(
    lambda k : P[k//2 + k%2]*B[k//2], 
    P.base)
Q = Q.change_base(Q.base.base().extend_variables('t').fraction_field())
t = Q.base("t"); q=Q.q
q_k = B.compatibility("q_n").change_base(Q.base)[0,0].extra_info()["variables"][0]

### Extending compatibility of $q^n$

In [4]:
P.compatibility("q_n")

Compatibility condition (0, 1, 1) with following coefficient matrix:
[0 1]

In [5]:
B.compatibility("q_n")

Compatibility condition (0, 1, 1) with following coefficient matrix:
[          q_k q*q_k^2 - q_k]

In [6]:
Q.set_compatibility("q_n", Compatibility([
    [P.compatibility("q_n").change_base(Q.base)[0,0], P.compatibility("q_n").change_base(Q.base)[0,1]],
    [B.compatibility("q_n").change_base(Q.base)[0,0], B.compatibility("q_n").change_base(Q.base)[0,1]]], 
    0, 1, 2), True, "any")

Compatibility condition (0, 1, 2) with following coefficient matrix:
[            0             1]
[          q_k q*q_k^2 - q_k]

In [7]:
check_compatibility(Q, "q_n", lambda seq: Qn*seq, 20)

True

### Extending compatibility of $E$

In [8]:
P.compatibility("E")

Compatibility condition (0, 0, 1) with following coefficient matrix:
[q_k]

In [9]:
B.compatibility("E")

Compatibility condition (1, 0, 1) with following coefficient matrix:
[  1 q_k]

In [10]:
coeffs = [
    [0, q_k, q_k**2],
    [q_k**2, q_k**2*(q_k-1), q_k**2*q]
]
coeffs = [[QRationalSequence(el, variables=[q_k], universe=Qn.universe, q=Qn.q) for el in row] for row in coeffs]

In [11]:
Q.set_homomorphism("E", Compatibility(coeffs, 2, 0, 2), True)

Compatibility condition (2, 0, 2) with following coefficient matrix:
[            0           q_k         q_k^2]
[        q_k^2 q_k^3 - q_k^2       q*q_k^2]

In [12]:
check_compatibility(Q, "E", lambda seq : seq.shift(), 20)

True

### Applying to example

To process the input from Ali we need to do the following:
* For "t", "q" and "qm" as e, do search and replace the regex "(e)(\d+)" -> "e^$2"
* Search and replace "qm" -> "q_n"
* make it a list removing the "g[i+n]"

In [13]:
op = "+".join([
    "-q^22*q_n^7*(-1+q*q_n)*(-1+q^2*q_n)*(-1+q^3*q_n)*(-1+q^2*q_n+q^3*q_n+q^4*q_n-q^5*q_n^2-q^6*q_n^2-q^7*q_n^2+q^9*q_n^3+q^12*q_n^5*t-q^15*q_n^5*t-2*q^16*q_n^5*t-q^17*q_n^5*t-q^18*q_n^5*t-q^16*q_n^6*t+q^18*q_n^6*t+2*q^19*q_n^6*t+2*q^20*q_n^6*t+q^21*q_n^6*t-q^22*q_n^7*t-q^23*q_n^7*t+q^30*q_n^10*t^2)", # g[n]
    "+q^22*q_n^7*(-1+q^2*q_n)*(-1+q^3*q_n)*(1-q*q_n-q^2*q_n-q^3*q_n-q^4*q_n+q^3*q_n^2+q^4*q_n^2+2*q^5*q_n^2+q^6*q_n^2+q^7*q_n^2-q^6*q_n^3-q^7*q_n^3-q^8*q_n^3-q^9*q_n^3-q^5*q_n^4*t+q^8*q_n^4*t+q^9*q_n^4*t-q^13*q_n^4*t-q^14*q_n^4*t+q^8*q_n^5*t+q^9*q_n^5*t-q^10*q_n^5*t-q^11*q_n^5*t-2*q^12*q_n^5*t-q^13*q_n^5*t+2*q^15*q_n^5*t+2*q^16*q_n^5*t+2*q^17*q_n^5*t+q^18*q_n^5*t-q^12*q_n^6*t+q^13*q_n^6*t+q^14*q_n^6*t+q^15*q_n^6*t-2*q^17*q_n^6*t-3*q^18*q_n^6*t-3*q^19*q_n^6*t-2*q^20*q_n^6*t-q^21*q_n^6*t-q^17*q_n^7*t+q^19*q_n^7*t+2*q^20*q_n^7*t+2*q^21*q_n^7*t+2*q^22*q_n^7*t+q^23*q_n^7*t-q^23*q_n^8*t-q^24*q_n^8*t-q^21*q_n^9*t^2-q^22*q_n^9*t^2-q^23*q_n^9*t^2+q^26*q_n^9*t^2+q^27*q_n^9*t^2+q^28*q_n^9*t^2+q^25*q_n^10*t^2+q^26*q_n^10*t^2-q^28*q_n^10*t^2-q^29*q_n^10*t^2-q^30*q_n^10*t^2+q^31*q_n^11*t^2+q^10*q_n^4*(1+t))*E", # g[1+n]
    "-q^25*q_n^10*(-1+q^3*q_n)*t*(1+q+2*q^2+q^3+q^4-q^7-q^2*q_n-3*q^3*q_n-5*q^4*q_n-5*q^5*q_n-4*q^6*q_n-2*q^7*q_n+q^9*q_n+q^10*q_n+q^11*q_n+2*q^5*q_n^2+4*q^6*q_n^2+7*q^7*q_n^2+6*q^8*q_n^2+5*q^9*q_n^2+q^10*q_n^2-2*q^12*q_n^2-q^13*q_n^2-q^14*q_n^2-q^8*q_n^3-3*q^9*q_n^3-4*q^10*q_n^3-4*q^11*q_n^3-2*q^12*q_n^3+q^14*q_n^3+q^15*q_n^3+q^16*q_n^3+q^12*q_n^4+q^13*q_n^4+q^14*q_n^4-q^17*q_n^4-q^11*q_n^5*t-q^12*q_n^5*t-q^13*q_n^5*t+q^15*q_n^5*t+3*q^16*q_n^5*t+4*q^17*q_n^5*t+5*q^18*q_n^5*t+4*q^19*q_n^5*t+3*q^20*q_n^5*t+2*q^21*q_n^5*t+q^22*q_n^5*t+q^14*q_n^6*t+2*q^15*q_n^6*t+q^16*q_n^6*t-2*q^18*q_n^6*t-5*q^19*q_n^6*t-7*q^20*q_n^6*t-8*q^21*q_n^6*t-7*q^22*q_n^6*t-5*q^23*q_n^6*t-2*q^24*q_n^6*t-q^25*q_n^6*t-q^18*q_n^7*t-q^19*q_n^7*t+2*q^21*q_n^7*t+3*q^22*q_n^7*t+5*q^23*q_n^7*t+5*q^24*q_n^7*t+4*q^25*q_n^7*t+2*q^26*q_n^7*t+q^27*q_n^7*t-q^25*q_n^8*t-q^26*q_n^8*t-q^27*q_n^8*t-q^28*q_n^8*t-q^28*q_n^10*t^2-q^29*q_n^10*t^2-q^30*q_n^10*t^2-q^31*q_n^10*t^2-q^32*q_n^10*t^2+q^32*q_n^11*t^2+q^33*q_n^11*t^2)*E^2", # g[2+n]
    "-q^24*q_n^9*t*(1+q+q^2+q^3+q^4-2*q^2*q_n-3*q^3*q_n-4*q^4*q_n-4*q^5*q_n-3*q^6*q_n-2*q^7*q_n-q^8*q_n+q^4*q_n^2+3*q^5*q_n^2+5*q^6*q_n^2+6*q^7*q_n^2+5*q^8*q_n^2+4*q^9*q_n^2+2*q^10*q_n^2+q^11*q_n^2-q^7*q_n^3-2*q^8*q_n^3-4*q^9*q_n^3-4*q^10*q_n^3-3*q^11*q_n^3-2*q^12*q_n^3-q^13*q_n^3+q^11*q_n^4+q^12*q_n^4+q^13*q_n^4+q^14*q_n^4-q^9*q_n^5*t-q^10*q_n^5*t-q^11*q_n^5*t+q^13*q_n^5*t+3*q^14*q_n^5*t+4*q^15*q_n^5*t+5*q^16*q_n^5*t+4*q^17*q_n^5*t+3*q^18*q_n^5*t+2*q^19*q_n^5*t+q^20*q_n^5*t+q^21*q_n^5*t+q^12*q_n^6*t+2*q^13*q_n^6*t+2*q^14*q_n^6*t-3*q^16*q_n^6*t-6*q^17*q_n^6*t-9*q^18*q_n^6*t-9*q^19*q_n^6*t-9*q^20*q_n^6*t-6*q^21*q_n^6*t-4*q^22*q_n^6*t-2*q^23*q_n^6*t-q^24*q_n^6*t-q^16*q_n^7*t-q^17*q_n^7*t+3*q^19*q_n^7*t+5*q^20*q_n^7*t+7*q^21*q_n^7*t+7*q^22*q_n^7*t+6*q^23*q_n^7*t+4*q^24*q_n^7*t+2*q^25*q_n^7*t+q^26*q_n^7*t-q^22*q_n^8*t-2*q^23*q_n^8*t-2*q^24*q_n^8*t-2*q^25*q_n^8*t-q^26*q_n^8*t-q^27*q_n^8*t-q^23*q_n^10*t^2-q^24*q_n^10*t^2-q^25*q_n^10*t^2-q^26*q_n^10*t^2-q^27*q_n^10*t^2-q^28*q_n^10*t^2+q^31*q_n^10*t^2+2*q^32*q_n^10*t^2+q^33*q_n^10*t^2+q^34*q_n^10*t^2+q^27*q_n^11*t^2+q^28*q_n^11*t^2+q^29*q_n^11*t^2+q^30*q_n^11*t^2+q^31*q_n^11*t^2-q^33*q_n^11*t^2-2*q^34*q_n^11*t^2-2*q^35*q_n^11*t^2-q^36*q_n^11*t^2+q^36*q_n^12*t^2+q^37*q_n^12*t^2-q^41*q_n^15*t^3)*E^3", # g[3+n]
    "-q^24*q_n^8*t*(-1+q*q_n+q^2*q_n+q^3*q_n-q^3*q_n^2-q^4*q_n^2-q^5*q_n^2+q^6*q_n^3+q^7*q_n^5*t-q^10*q_n^5*t-2*q^11*q_n^5*t-q^12*q_n^5*t-q^13*q_n^5*t-q^10*q_n^6*t+q^12*q_n^6*t+2*q^13*q_n^6*t+2*q^14*q_n^6*t+q^15*q_n^6*t-q^15*q_n^7*t-q^16*q_n^7*t+q^20*q_n^10*t^2)*E^4" # g[4+n]
])

In [14]:
%time M = Q.recurrence(op, output="ore")

CPU times: user 1min 53s, sys: 68.9 ms, total: 1min 53s
Wall time: 1min 53s


**TODO**: Try to get the gcd of the first column with the operator that annihilates the result we want.

$$S_k - q^2q_k^{4}t$$

In [15]:
M[0,0].polynomial().monomials()

[Sk^15,
 Sk^14,
 Sk^13,
 Sk^12,
 Sk^11,
 Sk^10,
 Sk^9,
 Sk^8,
 Sk^7,
 Sk^6,
 Sk^5,
 Sk^4,
 Sk^3,
 Sk^2,
 Sk,
 1]

In [16]:
M[1,0].polynomial().monomials()

[Sk^16,
 Sk^15,
 Sk^14,
 Sk^13,
 Sk^12,
 Sk^11,
 Sk^10,
 Sk^9,
 Sk^8,
 Sk^7,
 Sk^6,
 Sk^5,
 Sk^4,
 Sk^3,
 Sk^2,
 Sk]

In [17]:
op1 = M[0,0].parent()(M[0,0].polynomial()/gcd(M[0,0].polynomial().coefficients()))
op2 = M[0,0].parent()(M[1,0].polynomial()/gcd(M[1,0].polynomial().coefficients()))

In [22]:
Sk, q_k, q, t = [M[0,0].parent()(el) for el in ("Sk", "q_k", "q", "t")]

In [23]:
AA = Sk - q^2*q_k^4*t; AA

Sk - q^2*t*q_k^4

In [24]:
AA.gcrd(M[1,0])

Sk - q^2*t*q_k^4

In [25]:
AA.gcrd(M[0,0])

Sk - q^2*t*q_k^4

In [23]:
#%time op_gcd = M[0,0].gcrd(M[1,0]) # too long

## **Examples from [Sills' thesis](https://doi.org/10.37236/1706)**

### Identity 3.38

In the Identity 3.38 of Sills' thesis, they show that the solution to the $q$-recurrence equation
$$P_n = (1+q)P_{n-1} + (q^{2n}-q)P_n, \qquad P_0 = 1, P_1 = 1+q,$$
enjoys the following representation:
$$P_n = \sum_{j\geq 0} q^{2j(j+1)} \qbinom{n+1}{2j+1}.$$

With this software we can show this is correct:

In [19]:
B = QBinomialBasis(1,1)
M = B.recurrence("E^2 - (1+q)*E - (q_n**2*q**4 - q)", sections=2, output="ore")
OE, (q_n, E) = get_qshift_algebra("q_n", "q", "E", power=2, rational=False, base=B.base)
Msol = solution(E^2 - (1+q)*E - (q_n**2*q**4 - q), [1,1+q], name_q="q")
# We take the second column since we are taking the subset with `2j +1`.
# For the initial values, we use the method `inner_init_values` where "section" takes value 2 and "shift" 1
Mrec = M[0,1].gcrd(M[1,1])
c = solution(Mrec, B.inner_init_values(Msol, 1, Mrec, section=2, shift=1))
c[:10]

[1, q^4, q^12, q^24, q^40, q^60, q^84, q^112, q^144, q^180]

We can even check further that the sequence $c_k$ that we have created is indeed the coefficients appearing in the previous formula:

In [17]:
Sk = B.ore_gen(); q_k = B.ore_var(); q = B.q
true_c = solution(Sk - q_k**4*q**4, [1])
c == true_c

True

### Identity 3.39

In the identity 3.39, they consider a slightly different sequence $P_n$ defined now by:
$$P_n = (1+q)P_{n-1} + (q^{2n-2}-q)P_n, \qquad P_0 = 1, P_1 = 1,$$
and claimed that this sequences enjoys the following representation:
$$P_n = \sum_{j\geq 0} q^{2j^2}\qbinom{n}{2j}.$$

Once again, we can check this with this code:

In [34]:
B = QBinomialBasis()
M = B.recurrence("E^2 - (1+q)*E - (q_n**2*q**2 - q)", sections=2, output="ore")
OE, (q_n, E) = get_qshift_algebra("q_n", "q", "E", power=2, rational=False, base=B.base)
Msol = solution(E^2 - (1+q)*E - (q_n**2*q**2 - q), [1,1], name_q="q")

# We take the first column since we are taking the subset with `2j`.
# We take initial value 1 coming from P_0
c = solution(Mrec, B.inner_init_values(Msol, 1, Mrec, section=2))
c[:10]

[1, q^2, q^8, q^18, q^32, q^50, q^72, q^98, q^128, q^162]

And, as we did for the previous identity, we can check the equality of the sequences even further:

In [5]:
Sk = B.ore_gen(); q_k = B.ore_var(); q = B.q
true_c = solution(Sk - q_k**4*q**2, [1])
c == true_c

True

### Identity 3.5

In this case, we started with the recurrence obtained for the right-hand side of equation (3.5-b) and we want to see that this is equal to its left-hand side (which has a nice q-binomial basis expasion).

In [6]:
B = QBinomialBasis(1,1)
op = "+".join([
    "(-q^3+q^4*q_n^2+q^5*q_n^2-q^6*q_n^4)", #[n]
    "(q+q^2+q^3-q^4*q_n^2+q^5*q_n^3+q^7*q_n^3)*E", #[1+n]
    "(-1-q-q^2-q^5*q_n^2)*E^2", #[2+n]+
    "E^3", #[3+n]
])

In [7]:
M = B.recurrence(op, sections=2, output="ore")

In [8]:
M[0,1].gcrd(M[1,1])

Sk - q^3*q_k^4

And this recurrence is exactly the one needed for the proof:

In [9]:
solution(M[0,1].gcrd(M[1,1]), [1])[:10]

[1, q^3, q^10, q^21, q^36, q^55, q^78, q^105, q^136, q^171]

### A new example even more tricky

In this example from a Theorem of Berkovich and MacCoy, we use the following recurrence as starting point:

In [27]:
op = "+".join([
    "-q^4*q_n^2", #[n]
    "(-1-q^5*q_n^2)*E", #[1+n]
    "E^2", #[2+n]
]); op

'-q^4*q_n^2+(-1-q^5*q_n^2)*E+E^2'

Here we would like to find a recurrence for a sequence that can be written as:
$$a(n) = \sum_{k\geq 0} c_k \qbinomb{n}{k}{q^2}$$

In [28]:
B = QBinomialBasis(e=1)
out= B.recurrence(op, output="ore")

In [29]:
q = B.q
a = [1, 1+q^3, 1+q^3+q^4+q^5+q^8, 1+q^3+q^4+q^5+q^6+q^7+q^8+q^9+q^10+q^11+q^12+q^15]

In [30]:
a

[1,
 q^3 + 1,
 q^8 + q^5 + q^4 + q^3 + 1,
 q^15 + q^12 + q^11 + q^10 + q^9 + q^8 + q^7 + q^6 + q^5 + q^4 + q^3 + 1]

In [32]:
basis_matrix(B)

[                        1                         1                         1                         1                         1]
[                        0                         1                     q + 1               q^2 + q + 1         q^3 + q^2 + q + 1]
[                        0                         0                         1               q^2 + q + 1 q^4 + q^3 + 2*q^2 + q + 1]
[                        0                         0                         0                         1         q^3 + q^2 + q + 1]
[                        0                         0                         0                         0                         1]

In [33]:
out

Sk^4 + (-q^5*q_k^2 + (-4*q^5 + q + 1)*q_k - 4*q^5 + 2*q + 1)*Sk^3 + ((-q^5 - q^4 - q^3)*q_k^3 + (-6*q^5 - 6*q^4 - 5*q^3 + 1)*q_k^2 + (-12*q^5 - 12*q^4 - 8*q^3 + 3)*q_k - 8*q^5 - 8*q^4 - 4*q^3 + 2)*Sk^2 + ((-q^3 - q^2 - q)*q_k^4 + (-8*q^3 - 7*q^2 - 7*q)*q_k^3 + (-23*q^3 - 18*q^2 - 18*q)*q_k^2 + (-28*q^3 - 20*q^2 - 20*q)*q_k - 12*q^3 - 8*q^2 - 8*q)*Sk - 1/q*q_k^5 - 9/q*q_k^4 + ((q^2 - 32)/q)*q_k^3 + ((5*q^2 - 56)/q)*q_k^2 + ((8*q^2 - 48)/q)*q_k + (4*q^2 - 16)/q

In [ ]:
c = solution(out, basis_matrix(B, required_init(out)).solve_left(vector(a[:required_init(out)])))

In [37]:
for i in range(4):
    show(c[i].factor())

1

q^3

(q + 1) * q^5 * (q^2 - q + 1)

(q + 1) * q^5 * (q^2 - q + 1) * (q^7 + q^3 - 1)

In [ ]:
op2 = str(out).replace("Sk", "E").replace("q_k^2", "q_2n").replace("q_k^4", "q_2n^2").replace("q_k^6", "q_2n^3")

In [38]:
out2 = B.recurrence(op2, output="ore")

In [126]:
d = solution(out2, get_matrix(B, required_init(out2)).solve_left(vector(c[:required_init(out2)])))

In [39]:
for i in range(5):
    show(d[i].factor())

# <span style="color:green">TODO Things:</span>

* Allow the multiplication of a $q$-factorial basis by the $q$-Pochammer symbol $1/(q;q)_k$ -> It can be done now, but need some testing
* Allow the multiplication of a $q$-factorial basis by the $q$-Pochammer symbol $(-q;q)_k$ -> It can be done now, but need some testing

In [ ]:
B.is_hypergeometric(1/QPochhammerSequence(q,q))

(True, -1/q/(q_n - 1/q))

In [ ]:
B.is_hypergeometric(QPochhammerSequence(-q,q))

(True, q*q_n + 1)

In [59]:
B.scalar(QPochhammerSequence(-q,q)).__class__

<class 'pseries_basis.qbasis.qbasis.QScalarBasis'>